In [39]:
import pandas as pd
import datetime
import numpy as np
import time
from preprocessing_functions.functions_preprocessing_alarms import first_eboxes_preprocess, first_lights_preprocess, big_preprocess_eboxes, big_preprocess_lights
from preprocessing_functions.functions_preprocessing_readings import split_readings, pre_power, pre_power_peak, get_indexes_split, pre_readings_batches
from preprocessing_functions.functions_preprocessing_meteo import first_meteo_preprocess, meteo_groupby
from preprocessing_functions.functions_joins import join_light_alarms_readings_meteo, join_eboxes_alarms_readings_meteo
from preprocessing_functions.utils_preprocessing import return_monday

In [2]:
pd.options.mode.chained_assignment = None

# Pipleine test:

In [ ]:
municiplality = "canyelles"

meteo = pd.read_csv(f"/home/leibniz/Desktop/IHMAN/meteo_raw_data/meteo_{municiplality}.csv")
lights_alarms = pd.read_csv(f"/home/leibniz/Desktop/IHMAN/raw_data/{municiplality}_lights_alarms.csv")
eboxes_alarms = pd.read_csv(f"/home/leibniz/Desktop/IHMAN/raw_data/{municiplality}_eboxes_alarms.csv")
nodes = pd.read_csv(f"/home/leibniz/Desktop/IHMAN/raw_data/{municiplality}_nodes.csv")
readings = pd.read_csv(f"/home/leibniz/Desktop/IHMAN/raw_data/{municiplality}_readings.csv")

In [18]:
# Prepocess meteorological data:
meteo = first_meteo_preprocess(
    meteo = meteo,
    filter_dates = True,
    date_min = "2018-12-31",
    date_max = "2100-01-01"
)

meteo = meteo_groupby(meteo)

In [40]:
meteo

,Temp_max_max,Temp_avg_mean,Temp_avg_std,Temp_min_min,Dew_max_max,Dew_avg_mean,Dew_avg_std,Dew_min_min,Hum_max_max,Hum_avg_mean,...,Wind_avg_mean,Wind_avg_std,Wind_min_min,Pres_max_max,Pres_avg_mean,Pres_avg_std,Pres_min_min,Precipitation_mean,Precipitation_sum,dated
0,17.222222,8.785714,1.262969,2.777778,10.000000,2.634921,2.459729,-5.000000,93,67.485714,...,10.871429,0.930438,1,30.6,30.442857,0.097590,30.2,0.0,0.0,2015-01-01
1,17.222222,9.825397,1.715167,3.888889,12.222222,5.603175,2.086461,1.111111,94,75.728571,...,11.285714,3.068892,0,30.7,30.428571,0.179947,30.2,0.0,0.0,2015-01-08
2,15.000000,8.063492,1.462645,2.222222,12.222222,4.095238,2.499324,-1.111111,100,77.200000,...,9.785714,1.366783,0,30.2,29.885714,0.146385,29.7,0.0,0.0,2015-01-15
3,15.000000,8.500000,1.418934,2.222222,5.000000,-0.595238,3.009196,-13.888889,87,56.114286,...,11.271429,3.034092,0,30.3,30.057143,0.171825,29.7,0.0,0.0,2015-01-22
4,17.777778,8.777778,3.076193,2.222222,8.888889,2.111111,2.202692,-3.888889,93,65.085714,...,13.114286,3.745855,1,30.0,29.457143,0.214920,29.1,0.0,0.0,2015-01-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,18.888889,12.023810,1.523093,7.222222,13.888889,7.873016,1.477442,2.777778,100,76.500000,...,7.714286,1.145176,0,30.5,30.242857,0.127242,30.1,0.0,0.0,2022-12-29
418,17.222222,10.706349,1.953870,3.888889,12.222222,4.968254,1.631057,1.111111,88,69.385714,...,9.514286,1.475837,0,30.4,30.157143,0.171825,29.8,0.0,0.0,2023-01-05
419,16.111111,10.785714,1.563613,5.000000,12.222222,4.007937,3.081110,-7.222222,94,64.814286,...,12.400000,5.014645,0,30.4,29.900000,0.404145,29.2,0.0,0.0,2023-01-12
420,13.888889,6.619048,1.889006,-1.111111,6.111111,-2.388889,3.175912,-12.222222,81,54.871429,...,11.271429,3.436914,0,30.3,30.042857,0.151186,29.6,0.0,0.0,2023-01-19


In [19]:
# Readings preprocess:
powerReactive, powerActive, powerReactivePeak, powerActivePeak = split_readings(readings)
display(powerReactive)
powerReactive, powerActive = pre_power(powerReactive, "Reactive"), pre_power(powerActive, "Active")
display(powerReactive)
powerReactivePeak, powerActivePeak = pre_power_peak(powerReactivePeak, "Reactive"), pre_power_peak(powerActivePeak, "Active")

,id,dated,value,reading,component,componentid
1458542,AA000000D44C,2019-01-01 00:00:01,2370000,powerReactive,sum,0
1458543,AA000000D44C,2019-01-01 00:00:01,690000,powerReactive,p1,0
1458544,AA000000D44C,2019-01-01 00:00:01,1030000,powerReactive,p2,0
1458545,AA000000D44C,2019-01-01 00:00:01,650000,powerReactive,p3,0
1458546,AA000000D44C,2019-01-01 00:15:01,2340000,powerReactive,sum,0
...,...,...,...,...,...,...
7292687,AA000001169B,2022-10-06 07:30:01,0,powerReactive,p3,0
7292688,AA000001169B,2022-10-06 07:45:01,0,powerReactive,sum,0
7292689,AA000001169B,2022-10-06 07:45:01,0,powerReactive,p1,0
7292690,AA000001169B,2022-10-06 07:45:01,0,powerReactive,p2,0


,id,dated,powerReactive_sum,powerReactive_p1,powerReactive_p2,powerReactive_p3
0,AA000000D44C,2018-12-31,1.354618e+06,418489.583333,579010.416667,357100.694444
1,AA000000D44C,2019-01-07,1.411488e+06,457842.261905,567083.333333,387068.452381
2,AA000000D44C,2019-01-14,1.409673e+06,469687.500000,565535.714286,375208.333333
3,AA000000D44C,2019-01-21,1.384539e+06,455744.047619,554776.785714,373675.595238
4,AA000000D44C,2019-01-28,1.376741e+06,443898.809524,549122.023810,384404.761905
...,...,...,...,...,...,...
2162,AA000001169B,2022-09-05,6.023810e+01,51.928571,51.047619,71.095238
2163,AA000001169B,2022-09-12,4.861905e+01,37.547619,49.523810,76.785714
2164,AA000001169B,2022-09-19,5.313522e+01,41.485884,56.748886,68.255572
2165,AA000001169B,2022-09-26,5.760479e+01,40.455090,49.988024,68.910180


In [20]:
# Alarms preprocess
lights_alarms = first_lights_preprocess(lights_alarms)
eboxes_alarms = first_eboxes_preprocess(eboxes_alarms)

lights_alarms = big_preprocess_lights(lights_alarms)
eboxes_alarms = big_preprocess_eboxes(eboxes_alarms)

AA0000013912
AA0000013926
AA0000013931
AA000001393D
AA000001393E
AA0000016F53
AA000001726C
AA0000017324
AA00000184EB
AA0000018505
AA0000018696
AA0000018697
AA0000018698
AA0000018699
AA00000186B4
AA00000186D0
AA0000018704
AA0000018705
AA0000018706
AA0000018719
AA0000018745
AA0000018765
AA0000018769
AA000001876F
AA000001877F
AA0000018978
AA0000018981
AA00000189AC
AA00000189AD
AA00000189B8
AA00000189B9
AA00000189C6
AA00000189CB
AA00000189E7
AA0000018A0B
AA0000018A38
AA0000018A54
AA0000018A60
AA0000018A6C
AA0000018B4F
AA0000018B5F
AA0000018B99
AA0000018BA7
AA0000018BA8
AA0000018BB7
AA0000018C28
AA0000018C65
AA0000018C9B
AA0000018D53
AA0000018D6A
AA0000018D78
AA0000018D7B
AA0000018D85
AA0000018D87
AA0000018D89
AA0000018D8A
AA0000018D8C
AA0000018D93
AA0000018D9D
AA0000018DA1
AA0000018DA8
AA0000018DAB
AA0000018DB1
AA0000018DBD
AA0000018DD0
AA0000018DD8
AA0000018DDF
AA0000018DE3
AA0000018DEF
AA0000018DF7
AA0000018DFC
AA0000018E0C
AA0000018E10
AA0000018E9D
AA0000018EB4
AA0000018EC0
AA0000018ECB

In [21]:
# Join alarms, and nodes
lights_alarms_join = pd.merge(lights_alarms, nodes, on="id", how="left")

In [22]:
# You have to run the function join_eboxes_readings_meteo before runing join_light_alarms_readings_meteo
# because this last function modifies the datasets of the readings. In case you run the lights join
# first the code is going to return a Key error.
eboxes_alarms_join = join_eboxes_alarms_readings_meteo(
    eboxes_alarms = eboxes_alarms,
    eboxes_powerReactivePeak = powerReactivePeak,
    eboxes_powerReactive = powerReactive,
    eboxes_powerActive = powerActive,
    eboxes_powerActivePeak = powerActivePeak,
    meteo = meteo
)

lights_alarms_join = join_light_alarms_readings_meteo(
    light_errors = lights_alarms_join,
    eboxes_powerReactivePeak = powerReactivePeak,
    eboxes_powerReactive = powerReactive,
    eboxes_powerActive = powerActive,
    eboxes_powerActivePeak = powerActivePeak,
    meteo = meteo
)

In [23]:
print(len(lights_alarms_join))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(lights_alarms_join.isna().sum())

25993


id                                   0
week-4                               0
hours_week-4                         0
week-3                               0
hours_week-3                         0
week-2                               0
hours_week-2                         0
week-1                               0
hours_week-1                         0
current_week                         0
hours_current_week                   0
week+1                               0
hours_week+1                         0
week+2                               0
hours_week+2                         0
week+3                               0
hours_week+3                         0
week+4                               0
hours_week+4                         0
type                              2618
ebox_id                           2618
lat                               2618
lon                               2618
ReactivePeak_week-4               4488
ReactivePeak_week-3               4488
ReactivePeak_week-2      

In [24]:
print(len(eboxes_alarms_join))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(eboxes_alarms_join.isna().sum())

2321


id                                  0
week-4                              0
hours_week-4                        0
week-3                              0
hours_week-3                        0
week-2                              0
hours_week-2                        0
week-1                              0
hours_week-1                        0
current_week                        0
hours_current_week                  0
week+1                              0
hours_week+1                        0
week+2                              0
hours_week+2                        0
week+3                              0
hours_week+3                        0
week+4                              0
hours_week+4                        0
ReactivePeak_week-4               176
ReactivePeak_week-3               187
ReactivePeak_week-2               198
ReactivePeak_week-1               209
ReactivePeak_current_week         220
powerReactive_sum_week-4          176
powerReactive_p1_week-4           176
powerReactiv

# Readings spliting to be fit in memory

In [11]:
municipality = "illora"
data_dir = "/home/leibniz/Desktop/IHMAN"

readings_dates = pd.read_csv(f"{data_dir}/raw_data/{municipality}_readings_sorted.csv", usecols=["dated"]).reindex()

In [12]:
readings_dates["dated"] = pd.to_datetime(readings_dates["dated"])
start_date = return_monday(pd.to_datetime(readings_dates["dated"].min()))
end_date = return_monday(pd.to_datetime(readings_dates["dated"].max()) + pd.Timedelta(days=7))
range_dates = pd.date_range(start=start_date, end=end_date, freq="20W")

# Convert to timestamps and add the last date of the dataframe:
range_dates = [pd.Timestamp(return_monday(elem)) for elem in range_dates]
range_dates.append(pd.Timestamp(end_date))

In [13]:
# In this list we will store the indexes that will delimitate the batches:
spliting_indexes = np.array([])
for i in range(len(range_dates) -1):
    
    temp_df = readings_dates.loc[(range_dates[i] <= readings_dates["dated"]) & (readings_dates["dated"] < range_dates[i+1])]
    if len(temp_df) != 0:
        spliting_indexes = np.append(spliting_indexes, temp_df.index[-1])

# Add the index zero:
spliting_indexes = np.insert(spliting_indexes, 0, 0)

In [14]:
# Concatenated batches will be added to this dataframes:
powerReactive = pd.DataFrame()
powerActive = pd.DataFrame()
powerReactivePeak = pd.DataFrame()
powerActivePeak = pd.DataFrame()

# Name of the columns of the readings df:
readings_column_names = ["id", "dated", "value", "reading", "component", "componentid"]

# Convert to a list to iterate:
spliting_indexes_list = list(spliting_indexes)

# The readings preprocess must be done in batches because the datasets are too massive for some machines. In case that you 
# get a Memory error it is a good idea to take make larger the arg range_split_weeks_readings when running the code:
for i in range(len(spliting_indexes_list)-1):

    # For the first batch of date the function read_csv is able to read the names of the columns from the
    # first row of the csv file
    if i == 0:
        readings = pd.read_csv(
            f"{data_dir}/raw_data/{municipality}_readings_sorted.csv", 
            skiprows = int(spliting_indexes_list[i]), 
            nrows = int(spliting_indexes_list[i+1]-spliting_indexes_list[i]+1)
        )
    # For the other batches the function read_csv does not have acces to the column names so we have to
    # specify them
    else:
        readings = pd.read_csv(
            f"{data_dir}/raw_data/{municipality}_readings_sorted.csv",
            skiprows = int(spliting_indexes_list[i] + 2), 
            nrows = int(spliting_indexes_list[i+1]-spliting_indexes_list[i]),
            names = readings_column_names
        )

    powerReactive_, powerActive_, powerReactivePeak_, powerActivePeak_ = split_readings(readings)
    powerReactive_, powerActive_ = pre_power(powerReactive_, "Reactive"), pre_power(powerActive_, "Active")
    powerReactivePeak_, powerActivePeak_ = pre_power_peak(powerReactivePeak_, "Reactive"), pre_power_peak(powerActivePeak_, "Active")

    powerReactive = pd.concat([powerReactive, powerReactive_])
    powerActive = pd.concat([powerActive, powerActive_])
    powerReactivePeak = pd.concat([powerReactivePeak, powerReactivePeak_])
    powerActivePeak = pd.concat([powerActivePeak, powerActivePeak_])

In [4]:
meteo = pd.read_csv(f"/home/leibniz/Desktop/IHMAN/meteo_raw_data/meteo_{municiplality}.csv")

NameError: name 'municiplality' is not defined

In [32]:
len(readings_dates)

42724532

# Output datasets tests:

In [14]:
out_boxes = pd.read_csv("preprocessing_results/out_boxes.csv")
out_lights = pd.read_csv("preprocessing_results/out_lights.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'preprocessing_results/out_boxes.csv'

In [13]:
mejorada = out_lights.loc[out_lights["location"] == "mejorada"]
len(mejorada)

192280

In [11]:
raw_eboxes_alarms = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_eboxes_alarms.csv")
raw_lights_alarms = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_lights_alarms.csv")

# raw_nodes = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_nodes.csv")
# raw_readings = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_readings.csv")
# raw_readings_sorted = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_readings_sorted.csv")

/tmp/ipykernel_155716/1412182631.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_lights_alarms = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_lights_alarms.csv")


In [8]:
raw_meteo = pd.read_csv("/home/leibniz/Desktop/IHMAN/meteo_raw_data/new_meteo_mejorada.csv")

In [9]:
max(raw_meteo["Date"])

'2023-3-9'

In [10]:
min(raw_meteo["Date"])

'2014-1-1'

In [12]:
max(raw_lights_alarms["dated"])

'2036-01-21 21:49:44'

In [13]:
min(raw_lights_alarms["dated"])

'2014-09-10 18:51:55'

## alarms

In [26]:
raw_light_alarms = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_lights_alarms.csv")
len(raw_light_alarms)

/tmp/ipykernel_9059/2358804247.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_light_alarms = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_lights_alarms.csv")


5631094

In [27]:
first_lights_preprocess(raw_light_alarms)

,id,dated,alarm,flag
0,AA000000AB62,2015-03-24 18:22:12,lightcomm,off
1,AA000000AB62,2015-03-24 18:22:12,lighterr,off
2,AA000000AB63,2020-04-19 18:58:20,lighterr,on
3,AA000000AB63,2020-04-19 18:58:20,lighterr,on
4,AA000000AB63,2020-04-19 18:58:45,lighterr,off
...,...,...,...,...
196843,AA00000195F9,2023-04-02 18:43:16,lighterr,off
196844,AA00000195F9,2023-04-03 05:50:01,lighterr,on
196845,AA00000195F9,2023-04-03 05:50:01,lighterr,on
196846,AA00000195F9,2023-04-03 18:44:16,lighterr,off


In [1]:
test_date = "2022/06/09"

In [2]:
test_date[0:4]

'2022'

In [3]:
test_date[5:7]

'06'

In [5]:
test_date[8:]

'09'

In [1]:
from scrapy_meteo.scrapy_meteo import return_years_months

In [4]:
return_years_months(min_date="2014-04-01", max_date="2023-01-01")

4
2014
in!
[4, 5, 6, 7, 8, 9, 10, 11, 12]
2015
2016
2017
2018
2019
2020
2021
2022
2023


{2014: [4, 5, 6, 7, 8, 9, 10, 11, 12],
 2015: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 2016: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 2017: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 2018: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 2019: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 2020: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 2021: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 2022: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 2023: [1]}

In [3]:
from datetime import date
date.today().year

2023

In [4]:
date.today().month

5

In [5]:
import ast
ast.literal_eval("{'illora': ('2015-01-01', '2023-01-01'), 'canyelles': ('2015-01-01', '2023-01-01'), 'mejorada': ('2014-01-01', '2023-01-01')}")


{'illora': ('2015-01-01', '2023-01-01'),
 'canyelles': ('2015-01-01', '2023-01-01'),
 'mejorada': ('2014-01-01', '2023-01-01')}

In [18]:
meteo = pd.read_csv("/home/leibniz/Desktop/IHMAN/meteo_raw_data/new_meteo_mejorada.csv")
lights_alarms = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_lights_alarms.csv")

meteo = meteo_groupby(first_meteo_preprocess(meteo))
lights_alarms = 

/tmp/ipykernel_273039/815741648.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  lights_alarms = pd.read_csv("/home/leibniz/Desktop/IHMAN/raw_data/mejorada_lights_alarms.csv")


In [3]:
first_meteo_preprocess(meteo)

,Temp_max,Temp_avg,Temp_min,Dew_max,Dew_avg,Dew_min,Hum_max,Hum_avg,Hum_min,Wind_max,Wind_avg,Wind_min,Pres_max,Pres_avg,Pres_min,Precipitation,Date
0,10.000000,8.277778,7.222222,8.888889,7.777778,6.111111,100,96.7,87,10,7.1,3,28.1,28.0,28.0,0.0,2014-01-01
1,12.777778,11.000000,8.888889,12.222222,10.277778,8.888889,100,96.2,94,18,11.7,6,28.0,27.9,27.9,0.0,2014-01-02
2,12.777778,11.944444,11.111111,12.222222,10.444444,8.888889,100,91.1,82,15,10.2,3,28.1,28.1,28.0,0.0,2014-01-03
3,11.111111,8.722222,5.000000,11.111111,4.666667,-17.777778,100,81.1,57,26,16.4,5,28.1,27.9,27.8,0.0,2014-01-04
4,8.888889,6.388889,3.888889,7.777778,3.666667,-17.777778,100,86.5,65,14,7.8,5,28.2,28.2,28.1,0.0,2014-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3372,20.000000,12.611111,5.000000,0.000000,-1.666667,-6.111111,70,41.2,17,7,2.8,0,28.1,28.1,28.0,0.0,2023-03-27
3373,23.888889,13.611111,2.777778,0.000000,-2.722222,-5.000000,70,36.7,14,7,3.1,0,28.1,28.1,28.1,0.0,2023-03-28
3374,26.111111,16.111111,6.111111,3.888889,1.277778,0.000000,66,40.8,18,13,5.3,1,28.1,28.1,28.0,0.0,2023-03-29
3375,22.777778,16.944444,7.777778,8.888889,4.777778,1.111111,71,46.4,27,22,10.1,0,28.1,28.1,28.0,0.0,2023-03-30


In [6]:
meteo

,Unnamed: 0,Temp_max,Temp_avg,Temp_min,Dew_max,Dew_avg,Dew_min,Hum_max,Hum_avg,Hum_min,Wind_max,Wind_avg,Wind_min,Pres_max,Pres_avg,Pres_min,Precipitation,Date
0,0,50,46.9,45,48,46.0,43,100,96.7,87,10,7.1,3,28.1,28.0,28.0,0.0,2014-1-1
1,1,55,51.8,48,54,50.5,48,100,96.2,94,18,11.7,6,28.0,27.9,27.9,0.0,2014-1-2
2,2,55,53.5,52,54,50.8,48,100,91.1,82,15,10.2,3,28.1,28.1,28.0,0.0,2014-1-3
3,3,52,47.7,41,52,40.4,0,100,81.1,57,26,16.4,5,28.1,27.9,27.8,0.0,2014-1-4
4,4,48,43.5,39,46,38.6,0,100,86.5,65,14,7.8,5,28.2,28.2,28.1,0.0,2014-1-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3372,26,68,54.7,41,32,29.0,21,70,41.2,17,7,2.8,0,28.1,28.1,28.0,0.0,2023-3-27
3373,27,75,56.5,37,32,27.1,23,70,36.7,14,7,3.1,0,28.1,28.1,28.1,0.0,2023-3-28
3374,28,79,61.0,43,39,34.3,32,66,40.8,18,13,5.3,1,28.1,28.1,28.0,0.0,2023-3-29
3375,29,73,62.5,46,48,40.6,34,71,46.4,27,22,10.1,0,28.1,28.1,28.0,0.0,2023-3-30


In [34]:
lights_alarms["dated"].min()

'2014-09-10 18:51:55'

In [35]:
lights_alarms["dated"].max()

'2036-01-21 21:49:44'

In [22]:
preprocessed_lights = pd.read_csv("/home/leibniz/Desktop/IHMAN/preprocessing_results/out_lights.csv")

In [23]:
len(preprocessed_lights)

314413

In [24]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(preprocessed_lights.isna().sum())

Unnamed: 0                             0
id                                     0
week-4                                 0
hours_week-4                           0
week-3                                 0
hours_week-3                           0
week-2                                 0
hours_week-2                           0
week-1                                 0
hours_week-1                           0
current_week                           0
hours_current_week                     0
week+1                                 0
hours_week+1                           0
week+2                                 0
hours_week+2                           0
week+3                                 0
hours_week+3                           0
week+4                                 0
hours_week+4                           0
type                               48642
ebox_id                            57189
lat                                48642
lon                                48642
ReactivePeak_wee

In [8]:
clean = preprocessed_lights.loc[preprocessed_lights["Precipitation_sum_current_week"].isna()]

In [9]:
clean["current_week"].min()

'2014-10-06'

In [10]:
clean["current_week"].max()

'2015-06-01'

In [17]:
clean["current_week"].min()

'2014-10-06'

# Fuck it! Let's go step by step to see where the code is failing:

In [5]:
data_dir = "/home/leibniz/Desktop/IHMAN"
municipality = "canyelles"
n_weeks = 20

In [8]:
for municipality in ["canyelles", "illora", "mejorada"]:
    meteo = pd.read_csv(f"{data_dir}/meteo_raw_data/new_meteo_{municipality}.csv")
    
    print(municipality)
    print(meteo["Date"].min())
    print(meteo["Date"].max())

canyelles
2015-1-1
2023-4-9
illora
2015-1-1
2023-4-9
mejorada
2014-1-1
2023-4-9


In [9]:
meteo = first_meteo_preprocess(meteo=meteo, filter_dates=True, date_min="2013-12-20", date_max="2023-12-25")
meteo = meteo_groupby(meteo)

In [10]:
powerReactive, powerActive, powerReactivePeak, powerActivePeak = pre_readings_batches(splitting_indexes, municipality, data_dir)

In [11]:
lights_alarms = first_lights_preprocess(lights_alarms)
eboxes_alarms = first_eboxes_preprocess(eboxes_alarms)

In [12]:
lights_alarms = big_preprocess_lights(lights_alarms)
eboxes_alarms = big_preprocess_eboxes(eboxes_alarms)

AA0000013912
AA0000013926
AA0000013931
AA000001393D
AA000001393E
AA0000016F53
AA000001726C
AA0000017324
AA00000184EB
AA0000018505
AA0000018696
AA0000018697
AA0000018698
AA0000018699
AA00000186B4
AA00000186D0
AA0000018704
AA0000018705
AA0000018706
AA0000018719
AA0000018745
AA0000018765
AA0000018769
AA000001876F
AA000001877F
AA0000018978
AA0000018981
AA00000189AC
AA00000189AD
AA00000189B8
AA00000189B9
AA00000189C6
AA00000189CB
AA00000189E7
AA0000018A0B
AA0000018A38
AA0000018A54
AA0000018A60
AA0000018A6C
AA0000018B4F
AA0000018B5F
AA0000018B99
AA0000018BA7
AA0000018BA8
AA0000018BB7
AA0000018C28
AA0000018C65
AA0000018C9B
AA0000018D53
AA0000018D6A
AA0000018D78
AA0000018D7B
AA0000018D85
AA0000018D87
AA0000018D89
AA0000018D8A
AA0000018D8C
AA0000018D93
AA0000018D9D
AA0000018DA1
AA0000018DA8
AA0000018DAB
AA0000018DB1
AA0000018DBD
AA0000018DD0
AA0000018DD8
AA0000018DDF
AA0000018DE3
AA0000018DEF
AA0000018DF7
AA0000018DFC
AA0000018E0C
AA0000018E10
AA0000018E9D
AA0000018EB4
AA0000018EC0
AA0000018ECB

In [13]:
lights_alarms = pd.merge(lights_alarms, nodes, on="id", how="left")

In [14]:
lights_alarms = join_light_alarms_readings_meteo(
    light_errors = lights_alarms,
    eboxes_powerReactivePeak = powerReactivePeak,
    eboxes_powerReactive = powerReactive,
    eboxes_powerActive = powerActive,
    eboxes_powerActivePeak = powerActivePeak,
    meteo = meteo
)

In [15]:
lights_alarms.isna().sum()

id                                0
week-4                            0
hours_week-4                      0
week-3                            0
hours_week-3                      0
                                 ..
Pres_avg_avg_current_week         0
Pres_avg_std_current_week         0
Pres_min_min_current_week         0
Precipitation_avg_current_week    0
Precipitation_sum_current_week    0
Length: 183, dtype: int64

In [ ]:
len(lights_alarms)

25993

In [5]:
meteo

,Temp_max_max,Temp_avg_mean,Temp_avg_std,Temp_min_min,Dew_max_max,Dew_avg_mean,Dew_avg_std,Dew_min_min,Hum_max_max,Hum_avg_mean,...,Wind_avg_mean,Wind_avg_std,Wind_min_min,Pres_max_max,Pres_avg_mean,Pres_avg_std,Pres_min_min,Precipitation_mean,Precipitation_sum,dated
0,17.222222,8.785714,1.262969,2.777778,10.000000,2.634921,2.459729,-5.000000,93,67.485714,...,10.871429,0.930438,1,30.6,30.442857,0.097590,30.2,0.0,0.0,2015-01-01
1,17.222222,9.825397,1.715167,3.888889,12.222222,5.603175,2.086461,1.111111,94,75.728571,...,11.285714,3.068892,0,30.7,30.428571,0.179947,30.2,0.0,0.0,2015-01-08
2,15.000000,8.063492,1.462645,2.222222,12.222222,4.095238,2.499324,-1.111111,100,77.200000,...,9.785714,1.366783,0,30.2,29.885714,0.146385,29.7,0.0,0.0,2015-01-15
3,15.000000,8.500000,1.418934,2.222222,5.000000,-0.595238,3.009196,-13.888889,87,56.114286,...,11.271429,3.034092,0,30.3,30.057143,0.171825,29.7,0.0,0.0,2015-01-22
4,17.777778,8.777778,3.076193,2.222222,8.888889,2.111111,2.202692,-3.888889,93,65.085714,...,13.114286,3.745855,1,30.0,29.457143,0.214920,29.1,0.0,0.0,2015-01-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,18.888889,12.023810,1.523093,7.222222,13.888889,7.873016,1.477442,2.777778,100,76.500000,...,7.714286,1.145176,0,30.5,30.242857,0.127242,30.1,0.0,0.0,2022-12-29
418,17.222222,10.706349,1.953870,3.888889,12.222222,4.968254,1.631057,1.111111,88,69.385714,...,9.514286,1.475837,0,30.4,30.157143,0.171825,29.8,0.0,0.0,2023-01-05
419,16.111111,10.785714,1.563613,5.000000,12.222222,4.007937,3.081110,-7.222222,94,64.814286,...,12.400000,5.014645,0,30.4,29.900000,0.404145,29.2,0.0,0.0,2023-01-12
420,13.888889,6.619048,1.889006,-1.111111,6.111111,-2.388889,3.175912,-12.222222,81,54.871429,...,11.271429,3.436914,0,30.3,30.042857,0.151186,29.6,0.0,0.0,2023-01-19


In [9]:
light_alarms = pd.read_csv("/home/leibniz/Desktop/IHMAN/preprocessing_results/out_lights.csv")

In [10]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(light_alarms.isna().sum())

Unnamed: 0                             0
id                                     0
week-4                                 0
hours_week-4                           0
week-3                                 0
hours_week-3                           0
week-2                                 0
hours_week-2                           0
week-1                                 0
hours_week-1                           0
current_week                           0
hours_current_week                     0
week+1                                 0
hours_week+1                           0
week+2                                 0
hours_week+2                           0
week+3                                 0
hours_week+3                           0
week+4                                 0
hours_week+4                           0
type                               48642
ebox_id                            57189
lat                                48642
lon                                48642
ReactivePeak_wee

In [29]:
nans_light = light_alarms.loc[light_alarms["Pres_min_min_current_week"].isna()]

In [144]:
eboxes_top

Timestamp('2020-02-03 00:00:00')

In [32]:
nans_light["current_week"].max()

'2023-03-06'

### I need the last 4 weeks of readings, errors and all the nodes:

In [189]:
data_dir = "/home/leibniz/Desktop/IHMAN"
municipality = "mejorada"
# Get the last 4 weeks of alarms, readings and the nodes

eboxes_alarms = pd.read_csv(f"{data_dir}/raw_data/{municipality}_eboxes_alarms.csv")
light_alarms = pd.read_csv(f"{data_dir}/raw_data/{municipality}_lights_alarms.csv", usecols=lambda column: column != "info_value")
eboxes_alarms = first_eboxes_preprocess(eboxes_alarms)
light_alarms = first_lights_preprocess(light_alarms)

eboxes_alarms["dated"] = pd.to_datetime(eboxes_alarms["dated"])
light_alarms["dated"] = pd.to_datetime(light_alarms["dated"])

print(light_alarms["dated"].max())
# eboxes: 30-12-2019 to 02-02-2020
# lights: 27-02-2023 to 02-04-2023
eboxes_low = pd.to_datetime("2019-12-30")
eboxes_top = pd.to_datetime("2020-02-02")
light_low = pd.to_datetime("2023-02-27")
light_top = pd.to_datetime("2023-04-02")

eboxes_alarms = eboxes_alarms.loc[(eboxes_low <= eboxes_alarms["dated"]) & (eboxes_alarms["dated"] < eboxes_top)]
light_alarms = light_alarms.loc[(light_low <= light_alarms["dated"]) & (light_alarms["dated"] < light_top)]

2023-04-03 23:00:11


In [190]:
# Now just load the whole mejorada meteo data and get the dates from the ranges we need:
meteo = pd.read_csv("/home/leibniz/Desktop/IHMAN/meteo_raw_data/new_meteo_mejorada.csv")
meteo["Date"] = pd.to_datetime(meteo["Date"])

eboxes_meteo = meteo.loc[(eboxes_low <= meteo["Date"]) & (meteo["Date"] < eboxes_top)]
light_meteo = meteo.loc[(light_low <= meteo["Date"]) & (meteo["Date"] < light_top)]

In [191]:
# Store this dataframes to preprocess them in the predict_main.py module:
eboxes_alarms.to_csv("/home/leibniz/Desktop/IHMAN/data_to_predict/eboxes_alarms_to_predict.csv")
light_alarms.to_csv("/home/leibniz/Desktop/IHMAN/data_to_predict/light_alarms_to_predict.csv")

eboxes_meteo.to_csv("/home/leibniz/Desktop/IHMAN/data_to_predict/eboxes_meteo_to_predict.csv")
light_meteo.to_csv("/home/leibniz/Desktop/IHMAN/data_to_predict/light_meteo_to_predict.csv")

# Predictions test:

In [8]:
import pickle
import pandas as pd
import time
from predictive_models.predictive_models_functions import adboc_predict
from preprocessing_functions.functions_preprocessing_alarms import first_eboxes_preprocess, first_lights_preprocess, big_preprocess_eboxes, big_preprocess_lights
from preprocessing_functions.functions_preprocessing_readings import split_readings, pre_power, pre_power_peak, get_indexes_split, pre_readings_batches
from preprocessing_functions.functions_preprocessing_meteo import first_meteo_preprocess, meteo_groupby
from preprocessing_functions.functions_joins import join_light_alarms_readings_meteo, join_eboxes_alarms_readings_meteo
from preprocessing_functions.utils_preprocessing import return_monday

pd.options.mode.chained_assignment = None

In [9]:
def first_eboxes_preprocess(eboxes_alarms: pd.DataFrame, for_predicting = False, min_date = "1982-01-04", max_date = "2023-04-04") -> pd.DataFrame:
    """
    Performs basic preprocessing on the raw data of alarms such as filtering the errors that
    we want to predict ("lightcomm", "lighterr"), remove useless columns, parse dates and 
    eliminate dates out of a defined range.
    """

    eboxes_alarms = eboxes_alarms.loc[eboxes_alarms["subtype"] == "mcbpower"]

    eboxes_alarms = eboxes_alarms[["id", "dated", "subtype", "flag"]]

    eboxes_alarms["dated"] = pd.to_datetime(eboxes_alarms["dated"])

    if not for_predicting:
        # We have to do some filtering for the raw data because in the datasets for training we have some nonsense data
        # like data from the future.
        eboxes_alarms = eboxes_alarms.loc[(min_date <= eboxes_alarms["dated"]) & (eboxes_alarms["dated"] <= max_date)]

    return eboxes_alarms

def big_preprocess_eboxes(eboxes_alarms: pd.DataFrame, predicting_min_date: str = None, predicting_max_date: str = None, for_predicting: bool = False) -> pd.DataFrame:
    """
    The function converts the data of the alarms into a dataframe that is usable for training the models.
    Review the coments on the code to understand step by step what the code does.
    """
    
    start_time = time.time()

    # We create a dataframe of each one of the ids and we will clean the alarms. The final objective is to calculate
    # the percentage of time the ebox is not working wich is the time that passes from an alarm "off" utill it is 
    # turned "on". In the dataframe we have cases in wich there are two consecutive "on" or "off" alarms so we have
    # filter and just keep the first "on" and the first "off" in this case so we get the real time the ebox has been
    # not functioning. In the case that the alarm is turned "on" during a week and it is not turned "off" untill the next
    # week we will insert a fake "off" alarm at the last moment of the week and we will turn it "on" again in the first moment
    # of the following week.

    # The first step will be to dowload the data:
    # Read recipe inputs
    df = eboxes_alarms.copy()

    # We will reuse the code of the light alarms for the eboxes: The main thing that we have to modify for the eboxes
    # is that for the alarm subtype brdpower the flag "off" means that the ebox is suffering a breakdown and untill the
    # alarm turns "on" we will consider that the ebox has been down. This is exactly opposite to what happened with the
    # lamposts.

    # The first step is to change the name of the column subtype. Even tough this columns means a subtype of alarm, in this
    # code and from now on we will rename it as alarm so we are able to replicate a similar code as the one used for the lights
    df.rename(columns={"subtype": "alarm"}, inplace=True)

    # There are four different flags in this case: {off, on, offM, onM} depending if the error has been detected digitaly or manualy.
    # we do not care so we fill change this categories to {on off}:
    df["flag"] = df["flag"].replace(["onM"], "on")
    df["flag"] = df["flag"].replace(["offM"], "off")

    # As we already said the flags in the eboxes are opposite as the one in the lamposts, so to be able to use the same
    # code we will just change the values "off" to "on" and the "on" to "off" and then apply the same exact code that
    # we used for the lamposts.
    df["flag"] = df["flag"].replace({'on': 'offf', 'off': 'on'}).replace({"offf": "off"})

    # Sort by id and date of the alarm:
    df["dated"] = pd.to_datetime(df["dated"])
    df = df.sort_values(["id", "dated"])

    # Generate all the weeks from the first day we have data untill the last day. We will need this weeks later to
    # do a left join with the weeks that there are alarms

    # If we are preprocessing the data for doing predictions we will want to add the date_min and date max manualy,
    # otherwise the function will only consider the dates of the errors and you may end up with nan rows
    if for_predicting: 
        start_date = pd.to_datetime(predicting_min_date)
        end_date = pd.to_datetime(predicting_max_date)
    else:
        start_date = df["dated"].min()
        end_date = return_monday(df["dated"].max()) + pd.Timedelta(days=7)

    weeks = pd.date_range(start=start_date, end=end_date, freq='W').floor("D").strftime('%Y-%m-%d %H:%M:%S').tolist()
    # Transform to a dataframe to do operations later:
    weeks = pd.DataFrame(weeks, columns=["week"])
    weeks["week"] = pd.to_datetime(weeks["week"])
    weeks["week"] = weeks["week"].apply(lambda x: return_monday(x))

    # For each unique id we will implement all the code:
    # First create the dataframe where we will store all the resturned data:
    general_lag_dataframe = pd.DataFrame()

    # List of all the ids:
    ids_list = df["id"].unique()

    # Store the name of the columns in oder to later reordenate the final returned dataframe.
    if for_predicting:
        columns_order = ["id", "week-4", "hours_week-4", "week-3", "hours_week-3", "week-2", "hours_week-2", "week-1", "hours_week-1",
                        "current_week", "hours_current_week"]
    else:     
        columns_order = ["id", "week-4", "hours_week-4", "week-3", "hours_week-3", "week-2", "hours_week-2", "week-1", "hours_week-1",
                        "current_week", "hours_current_week", "week+1", "hours_week+1", "week+2", "hours_week+2", "week+3", "hours_week+3", "week+4", "hours_week+4"]

    # Generate a dataframe for each one of the ids and then apply all the transformations. Once done, add the data
    # to the dataframe general_lag_dataframe.
    for idd in ids_list:
        print(idd)
        
        # Dataframe of all the alarms for the id "idd":
        tt = df.loc[df["id"] == idd]
        
        #For each one of the elements in the list we have: elem[0] contains the week represented by sunday 
        # and elem[1] has the data in a dataframe
        grouped_weeks = list(tt.groupby(pd.Grouper(key="dated", freq="W")))
        # Here we will transform all the tuples to lists so we can change the elements after
        grouped_weeks = [list(elem) for elem in grouped_weeks]

        # Now we have to filtrate the empty dataframes that generates groupby for the weeks where there are no alarms
        grouped_weeks = [elem for elem in grouped_weeks if not elem[1].empty]

        # Eliminate the repeated flags such as "off" followed by "off" or "on" followed by "on" for the first element
        # of the list grouped_weeks. We do it because in the iteration we will not consider it so we have to do it now
        first_week_data = grouped_weeks[0][1]
        first_week_data["prev_flag"] = first_week_data["flag"].shift()

        first_week_data = first_week_data.loc[
            ((first_week_data["flag"] == "on") & (first_week_data["prev_flag"] == "off")) |
            ((first_week_data["flag"] == "off") & (first_week_data["prev_flag"] == "on")) |
            ((first_week_data["flag"] == "off") & (first_week_data["prev_flag"].isna())) | # This two last comparisons are for the first rows because they don't have a previous flag
            ((first_week_data["flag"] == "on") & (first_week_data["prev_flag"].isna()))
        ]

        # We store it to the data of the first week:
        grouped_weeks[0][1] = first_week_data

        # Here we begin the iteration for all the other weeks:
        for i, (week, data) in enumerate(grouped_weeks[1:], 1): # We do not consider the first week because it has no previous week to check
            # Get the data and week from the previous week:
            previous_week, previous_week_data = grouped_weeks[i-1][0], grouped_weeks[i-1][1]

            # Eliminate the repeated flags such as "off" followed by "off" or "on" followed by "on"
            data["prev_flag"] = data["flag"].shift()
            data = data.loc[
                ((data["flag"] == "on") & (data["prev_flag"] == "off")) |
                ((data["flag"] == "off") & (data["prev_flag"] == "on")) |
                ((data["flag"] == "off") & (data["prev_flag"].isna())) | # This two last comparisons are for the first rows because they don't have a previous flag
                ((data["flag"] == "on") & (data["prev_flag"].isna()))
            ]

            # Data that we will need:
            last_moment_previous_week = previous_week + pd.Timedelta(hours=23, minutes=59, seconds=59) #We will need the last moment of the week
            first_moment_current_week = (week - pd.Timedelta(days=week.dayofweek)).replace(hour=0, minute=0, second=0) #We will need to the first moment of the current week

            #We have to get the last flag of the previous week
            last_flag_previous_week = previous_week_data.loc[previous_week_data.index[-1], "flag"]

            if last_flag_previous_week == "on":
                # If the last flag from the previous week is "on" then we have to set a new row on the previous week data
                # in the last position to set a flag "off". Then, in the current week data we will add a new row before the
                # first week to set again the alarm to "on"

                # Here we create the new row to set the alarm "off" in the previous week
                new_row_previous_week = pd.DataFrame(
                    {
                    "id": [idd],
                    "dated": [last_moment_previous_week],
                    "alarm": ["turn_off_end_week"], # We will put this in alarm to know witch alarms where inserted by us
                    "flag": ["off"],
                    "prev_flag": ["Empty"] # Empty because we do not need it anymore and it will help us identify this rows
                    }
                )

                # We create the new df with the new row at the very end of the week
                new_previous_week_data = pd.concat(
                    [previous_week_data, new_row_previous_week],
                    sort=True # Remove the warning of pd.concat
                )
                # We update the dataframe in the list grouped_weeks
                grouped_weeks[i-1][1] = new_previous_week_data

                # Now we have to set the flag "on" in the first moment of the current week:
                # Here we create the new row to set the alarm "on" in the current week
                new_row_current_week = pd.DataFrame(
                    {
                    "id": [idd],
                    "dated": [first_moment_current_week],
                    "alarm": ["turn_on_begining_week"], # We will put this in alarm to know witch alarms where inserted by us
                    "flag": ["on"],
                    "prev_flag": ["Empty"] # Empty because we do not need it anymore and it will help us identify this rows
                    }
                )

                # We create the new df with the new row at the very begining of the week
                new_current_week_data = pd.concat(
                    [new_row_current_week, data],
                    sort=True # Remove the warning of pd.concat
                )
                # We update the dataframe in the list grouped_weeks
                grouped_weeks[i][1] = new_current_week_data
            else:
                # Simply update the dataframe with the same but with removed rows that contain two identical flags in a row
                grouped_weeks[i][1] = data

        # Once this is done we have to check if the last alarm of the last week is "on". In this case we will add a row turning it off
        # in the last moment of the week:
        last_recorded_week, last_recorded_week_data = grouped_weeks[-1][0], grouped_weeks[-1][1]

        # Get the last flag from the last week
        last_flag = last_recorded_week_data["flag"].values[-1]
        # and get the last moment of the last week
        last_moment_last_recorded_week = last_recorded_week + pd.Timedelta(hours=23, minutes=59, seconds=59)

        if last_flag == "on":
            new_row_last_flag = pd.DataFrame(
                {
                "id": [idd],
                "dated": [last_moment_last_recorded_week],
                "alarm": ["turn_off_end_last_week"], # We will put this in alarm to know witch alarms where inserted by us
                "flag": ["off"],
                "prev_flag": ["Empty"] # Empty because we do not need it anymore and it will help us identify this rows
                }
            )

            last_recorded_week_data = pd.concat(
                [last_recorded_week_data, new_row_last_flag],
                sort=True
            )

            grouped_weeks[-1][1] = last_recorded_week_data

        # Now we have to concat all the dataframes from all the weeks contained in the list grouped_weeks in one big dataframe
        concatenated_weeks = pd.concat(
            [week_data[1] for week_data in grouped_weeks],
            sort=True # Remove the warning of pd.concat
        )

        # At this point there are some cases where we will still have two "on" alarms or two "off" alarms in a row. For example
        # in the case that we have two weeks in a row where we only have "on" alarms utill now the code is going to return the 
        # begining of the end of the previous week with "off", the beggining of the current week with "off" and before the "off"
        # of the end of the week we will still have an "on" alarm. This is caused because the deletion of the same alarms in a row is done 
        # before the add of the new rows in the beggining and end of the week

        # So let's eliminate this cases too:
        # first we have to frop the old prev_flag column that now is useless:
        concatenated_weeks.drop("prev_flag", axis=1, inplace=True)

        # and create the new one:
        concatenated_weeks["prev_flag_concat"] = concatenated_weeks["flag"].shift()

        concatenated_weeks = concatenated_weeks.loc[
            ((concatenated_weeks["flag"] == "on") & (concatenated_weeks["prev_flag_concat"] == "off")) |
            ((concatenated_weeks["flag"] == "off") & (concatenated_weeks["prev_flag_concat"] == "on")) |
            ((concatenated_weeks["flag"] == "off") & (concatenated_weeks["prev_flag_concat"].isna())) | # This two last comparisons are for the first rows because they don't have a previous flag
            ((concatenated_weeks["flag"] == "on") & (concatenated_weeks["prev_flag_concat"].isna()))
        ]

        # We have some weeks where there is no data and thus if this weeks are between an "on" and "off" flag they will not
        # appear on the dataframe. What we will do is a left join with the variable week generated at the begining
        # of the notebook to see the rows that do not appear. Then we will activate the alarm at the begining of the week
        # and deactivate it at the end of the same week:

        # Set the day to monday
        concatenated_weeks["week"] = concatenated_weeks["dated"].apply(lambda x: return_monday(x))
        # Left join with weeks to detect the missing weeks
        concatenated_weeks_merged = pd.merge(weeks, concatenated_weeks, on="week", how="left")

        # Store the result of the filling:
        filled_dataframe = pd.DataFrame()

        # We add the first row
        if not pd.isna(concatenated_weeks_merged.iloc[0]["alarm"]):
            first_new_row = pd.DataFrame(
                {
                    "week": [concatenated_weeks_merged.iloc[0]["week"]],
                    "alarm": [concatenated_weeks_merged.iloc[0]["alarm"]],
                    "dated": [concatenated_weeks_merged.iloc[0]["dated"]],
                    "flag": [concatenated_weeks_merged.iloc[0]["flag"]],
                    "id": [concatenated_weeks_merged.iloc[0]["id"]]
                }
            )

            filled_dataframe = pd.concat(
                [filled_dataframe, first_new_row],
                sort=True
            )

        # Begin the iteration where we will add one by one the rows to the dataframe filled_dataframe
        for i in range(1, len(concatenated_weeks_merged)-1):

            current_row = concatenated_weeks_merged.iloc[i]

            # If we find a normal row we add it to the dataframe
            if (current_row["alarm"] == "mcbpower") | (current_row["alarm"] == "turn_off_end_week") | (current_row["alarm"] == "turn_on_begining_week"):
                current_row["dated"] = pd.to_datetime(current_row["dated"])

                new_row = pd.DataFrame(
                    {
                        "week": [current_row["week"]],
                        "alarm": [current_row["alarm"]],
                        "dated": [current_row["dated"]],
                        "flag": [current_row["flag"]],
                        "id": [current_row["id"]]
                    }
                )
                filled_dataframe = pd.concat([filled_dataframe, new_row], sort=True)

            if not filled_dataframe.empty:
                # If the last row of the filled dataframe is a "turn_off_week" ant the current is a Emty we add the "on" and
                # off for the begining and the end of the week
                last_row_filled_dataframe = filled_dataframe.iloc[-1]

                if (pd.isna(current_row["alarm"])) & (last_row_filled_dataframe["alarm"] in ["turn_off_end_week", "turn_off_end_week_filled"]):

                    new_row_begining_dated = datetime.datetime.combine(current_row["week"], datetime.time(0,0,0))
                    new_row_begining_week = pd.DataFrame(
                        {
                            "week": [current_row["week"]],
                            "alarm": ["turn_on_begining_week_filled"],
                            "dated": [new_row_begining_dated], # Add the time to the date
                            "flag": ["on"],
                            "id": [idd]
                        }
                    )
                    # We have to add 6 days because the representative of the week in this case is 
                    new_row_end_dated = datetime.datetime.combine(current_row["week"]+ pd.Timedelta(days=6), datetime.time(23,59,59))
                    new_row_end_week = pd.DataFrame(
                        {
                            "week": [current_row["week"]],
                            "alarm": ["turn_off_end_week_filled"],
                            "dated": [new_row_end_dated], # In this case we add the time to represent the last moment of the week
                            "flag": ["off"],
                            "id": [idd]
                        }
                    )

                    filled_dataframe = pd.concat(
                        [filled_dataframe, new_row_begining_week, new_row_end_week],
                        sort=True
                    )

        # Again we have to do a left join with the dataframe weeks to detect the NaN values:
        filled_dataframe_merged = pd.merge(weeks, filled_dataframe, on="week", how="left")
        
        # Now for each one of the weeks we have to calculate the total time passed between an "on" alarm and an "off" alarm

        # In this dataframe we will store the amount of hours of a ebox that has been malfunctioning for each week
        week_hours_dataframe = pd.DataFrame()
        for week in weeks["week"]:
            # In this variable we will store the amount of hours for this week:
            total_hours = 0
            on_timestamp = None

            # Dataframe with the alarms of the week:
            week_alarms_dataframe = filled_dataframe_merged.loc[filled_dataframe_merged["week"] == week]
            # Iterate trough the df to count the hours:
            for _, row in week_alarms_dataframe.iterrows():
                if row["flag"] == "on":
                    on_timestamp = row["dated"]

                if (row["flag"] == "off") & (on_timestamp is not None):
                    total_hours += (row["dated"] - on_timestamp).total_seconds() / 3600

                    on_timestamp = None

            new_week_hours = pd.DataFrame(
                {
                    "id": [idd],
                    "week": [week],
                    "malfunctioning_hours": [total_hours] 
                }
            )

            week_hours_dataframe = pd.concat(
                [week_hours_dataframe, new_week_hours],
                sort=True
            )

        # Now we want to get the data in the format:
        # row = {"week-4": date_week_prev_4, "hours_week-4": hours_week_prev_4, ..., "week-1": date_week_prev_1, "hours_week-1": hours_week_prev_1, "current_week": date_current_week, "hours_current_week":  "week+1": date_week_next_1, "hours_week+1": hours_week_next_1 ..., "week+4": date_week_next_4, "hours_week+4": hours_week_next_4}

        # In this dataframe we will store the data in the format we have mentioned:
        lag_dataframe = pd.DataFrame()

        # Here we have to follow two different paths. The first one will be when preparing the data for training and the other one will be when 
        # preparing the data for predicting. We shall begin with predicting:

        if for_predicting:
            lag_dataframe = pd.DataFrame(
                {
                    "id": [idd],

                    "week-4": [week_hours_dataframe.iloc[0]["week"]],
                    "hours_week-4": [week_hours_dataframe.iloc[0]["malfunctioning_hours"]],

                    "week-3": [week_hours_dataframe.iloc[1]["week"]],
                    "hours_week-3": [week_hours_dataframe.iloc[1]["malfunctioning_hours"]],

                    "week-2": [week_hours_dataframe.iloc[2]["week"]],
                    "hours_week-2": [week_hours_dataframe.iloc[2]["malfunctioning_hours"]],

                    "week-1": [week_hours_dataframe.iloc[3]["week"]],
                    "hours_week-1": [week_hours_dataframe.iloc[3]["malfunctioning_hours"]],

                    "current_week": [week_hours_dataframe.iloc[4]["week"]],
                    "hours_current_week": [week_hours_dataframe.iloc[4]["malfunctioning_hours"]]
                }
            )

            # Reorder the dataframe so it is in the same order we have defined:
            lag_dataframe = lag_dataframe[columns_order]
            
            # Add the dataframe to the general one:
            general_lag_dataframe = pd.concat(
                [general_lag_dataframe, lag_dataframe],
                sort=True
            )

        else:
            # Begin the loop at 4 and end at -4 so we don't get the error: "Out of range"
            for i in range(4, len(weeks)-4):
                # Create the new row to add:
                to_add_row = pd.DataFrame(
                    {
                        "id": [idd],

                        "week-4": [week_hours_dataframe.iloc[i-4]["week"]],
                        "hours_week-4": [week_hours_dataframe.iloc[i-4]["malfunctioning_hours"]],

                        "week-3": [week_hours_dataframe.iloc[i-3]["week"]],
                        "hours_week-3": [week_hours_dataframe.iloc[i-3]["malfunctioning_hours"]],

                        "week-2": [week_hours_dataframe.iloc[i-2]["week"]],
                        "hours_week-2": [week_hours_dataframe.iloc[i-2]["malfunctioning_hours"]],

                        "week-1": [week_hours_dataframe.iloc[i-1]["week"]],
                        "hours_week-1": [week_hours_dataframe.iloc[i-1]["malfunctioning_hours"]],

                        "current_week": [week_hours_dataframe.iloc[i]["week"]],
                        "hours_current_week": [week_hours_dataframe.iloc[i]["malfunctioning_hours"]],

                        "week+1": [week_hours_dataframe.iloc[i+1]["week"]],
                        "hours_week+1": [week_hours_dataframe.iloc[i+1]["malfunctioning_hours"]],

                        "week+2": [week_hours_dataframe.iloc[i+2]["week"]],
                        "hours_week+2": [week_hours_dataframe.iloc[i+2]["malfunctioning_hours"]],

                        "week+3": [week_hours_dataframe.iloc[i+3]["week"]],
                        "hours_week+3": [week_hours_dataframe.iloc[i+3]["malfunctioning_hours"]],

                        "week+4": [week_hours_dataframe.iloc[i+4]["week"]],
                        "hours_week+4": [week_hours_dataframe.iloc[i+4]["malfunctioning_hours"]]
                    }
                )

                lag_dataframe = pd.concat(
                    [lag_dataframe, to_add_row],
                    sort=True,
                    ignore_index=True
                )

            # Reorder the dataframe so it is in the same order we have defined:
            lag_dataframe = lag_dataframe[columns_order]
            
            # Add the dataframe to the general one:
            general_lag_dataframe = pd.concat(
                [general_lag_dataframe, lag_dataframe],
                sort=True
            )
        
    # Reordenate with the list columns_order:
    general_lag_dataframe = general_lag_dataframe[columns_order]

    end_time = time.time()

    print("Execution time:" + str(end_time - start_time))

    return general_lag_dataframe

def change_format(date: str) -> str:
    """
    Get the date from a datetime object and convert it into a string
    """
    return str(date)[:10]

def join_eboxes_alarms_readings_meteo(
        eboxes_alarms: pd.DataFrame, 
        eboxes_powerReactivePeak: pd.DataFrame,
        eboxes_powerReactive: pd.DataFrame,
        eboxes_powerActive: pd.DataFrame,
        eboxes_powerActivePeak: pd.DataFrame,
        meteo: pd.DataFrame
    ) -> pd.DataFrame:
    """
    Joins the preprocessed data from light_alarms, readings, and meteo data by weeks
    """

    #Change meteo date format
    meteo['dated'] = meteo['dated'].apply(change_format)
    for week in ["week-4", "week-3", "week-2", "week-1", "current_week"]:
        eboxes_alarms[week] = eboxes_alarms[week].apply(lambda x: str(x))

    # joins for the dataset eboxes_powerReactivePeak:
    for week in ["week-4", "week-3", "week-2", "week-1", "current_week"]:

        eboxes_alarms = pd.merge(
            eboxes_alarms, 
            eboxes_powerReactivePeak.rename(columns={"dated": week}), 
            on=["id", week],
            how="left"
        )
        
        eboxes_alarms.rename(columns={"ReactivePeak": "ReactivePeak_" + week}, inplace=True)

    # joins for the dataset eboxes_powerReactive:
    for week in ["week-4", "week-3", "week-2", "week-1", "current_week"]:

        eboxes_alarms = pd.merge(
            eboxes_alarms, 
            eboxes_powerReactive.rename(columns={"dated": week}), 
            on=["id", week],
            how="left"
        )
        
        eboxes_alarms.rename(
            columns={
                "powerReactive_sum": "powerReactive_sum_" + week,
                "powerReactive_p1": "powerReactive_p1_" + week,
                "powerReactive_p2": "powerReactive_p2_" + week,
                "powerReactive_p3": "powerReactive_p3_" + week
                    },
            inplace=True)

    # joins for the dataset eboxes_powerActivePeak:
    for week in ["week-4", "week-3", "week-2", "week-1", "current_week"]:

        eboxes_alarms = pd.merge(
            eboxes_alarms, 
            eboxes_powerActivePeak.rename(columns={"dated": week}), 
            on=["id", week],
            how="left"
        )
        
        eboxes_alarms.rename(columns={"ActivePeak": "ActivePeak_" + week}, inplace=True)

    # joins for the dataset eboxes_powerActive:
    for week in ["week-4", "week-3", "week-2", "week-1", "current_week"]:

        eboxes_alarms = pd.merge(
            eboxes_alarms, 
            eboxes_powerActive.rename(columns={"dated": week}), 
            on=["id", week],
            how="left"
        )
        
        eboxes_alarms.rename(
            columns={
                "powerActive_sum": "powerActive_sum_" + week,
                "powerActive_p1": "powerActive_p1_" + week,
                "powerActive_p2": "powerActive_p2_" + week,
                "powerActive_p3": "powerActive_p3_" + week
                    },
            inplace=True)
        

    # joins for the dataset meteo_canyelles:
    for week in ["week-4", "week-3", "week-2", "week-1", "current_week"]:

        eboxes_alarms = pd.merge(
            eboxes_alarms, 
            meteo.rename(columns={"dated": week}), 
            on=[week],
            how="left"
        )
        
        eboxes_alarms.rename(
            columns={
                "Dew_max_max": "Dew_max_max_" + week,
                "Hum_min_min": "Hum_min_min_" + week,
                "Wind_max_max": "Wind_max_max_" + week,
                "Temp_max_max": "Temp_max_max_" + week,
                "Temp_min_min": "Temp_min_min_" + week,
                "Pres_min_min": "Pres_min_min_" + week,
                "Wind_avg_mean": "Wind_avg_avg_" + week,
                "Wind_avg_std": "Wind_avg_std_" + week,
                "Hum_avg_mean": "Hum_avg_avg_" + week,
                "Dew_avg_mean": "Dew_avg_avg_" + week,
                "Dew_avg_std": "Dew_avg_std_" + week,
                "Hum_max_max": "Hum_max_max_" + week,
                "Temp_avg_mean": "Temp_avg_avg_" + week,
                "Temp_avg_std": "Temp_avg_std_" + week,
                "Pres_max_max": "Pres_max_max_" + week,
                "Pres_avg_mean": "Pres_avg_avg_" + week,
                "Pres_avg_std": "Pres_avg_std_" + week,
                "Precipitation_mean": "Precipitation_avg_" + week,
                "Precipitation_sum": "Precipitation_sum_" + week,
                "Dew_min_min": "Dew_min_min_" + week,
                "Wind_min_min": "Wind_min_min_" + week,
                "Hum_avg_std": "Hum_avg_std_" + week
                    },
            inplace=True)
    
    return eboxes_alarms

In [10]:
eboxes_alarms = pd.read_csv("/home/leibniz/Desktop/IHMAN/data_to_predict/eboxes_alarms.csv")
light_alarms = pd.read_csv("/home/leibniz/Desktop/IHMAN/data_to_predict/lights_alarms.csv")
nodes = pd.read_csv("/home/leibniz/Desktop/IHMAN/data_to_predict/nodes.csv")
powerActive = pd.read_csv("/home/leibniz/Desktop/IHMAN/data_to_predict/powerActive.csv")
powerReactive = pd.read_csv("/home/leibniz/Desktop/IHMAN/data_to_predict/powerReactive.csv")
powerActivePeak = pd.read_csv("/home/leibniz/Desktop/IHMAN/data_to_predict/powerActivePeak.csv")
powerReactivePeak = pd.read_csv("/home/leibniz/Desktop/IHMAN/data_to_predict/powerActivePeak.csv")

# meteo here:
meteo = pd.read_csv("/home/leibniz/Desktop/IHMAN/data_to_predict/new_meteo_mejorada.csv")

predicting_min = "2023-04-17"
predicting_max = "2023-05-21"

In [11]:
# For the lights and eboxes that have alarms in the last 5 weeks we have to do the normal preprocessing:
# First the eboxes:
powerReactive, powerActive = pre_power(powerReactive, "Reactive"), pre_power(powerActive, "Active")
powerReactivePeak, powerActivePeak = pre_power_peak(powerReactivePeak, "Reactive"), pre_power_peak(powerActivePeak, "Active")

meteo = first_meteo_preprocess(
    meteo = meteo,
    filter_dates = False
)

meteo = meteo_groupby(meteo)

eboxes_alarms = first_eboxes_preprocess(eboxes_alarms, for_predicting=True)

eboxes_alarms = big_preprocess_eboxes(
    eboxes_alarms = eboxes_alarms,
    for_predicting = True,
    predicting_min_date = predicting_min,
    predicting_max_date = predicting_max
)

eboxes_alarms = join_eboxes_alarms_readings_meteo(
    eboxes_alarms = eboxes_alarms,
    eboxes_powerReactivePeak = powerReactivePeak,
    eboxes_powerReactive = powerReactive,
    eboxes_powerActive = powerActive,
    eboxes_powerActivePeak = powerActivePeak,
    meteo = meteo
)

# Untill this point we have just considered the eboxes that have had an error in the last 4 weeks. 
# We should consider too all the other alarms that have no errors in the last 4 weeks and put them
# in a dataframe format identical to the eboxes-alarms dataframe. This way we will be able to do predictionns
# for all the eboxes in the city.

# Let's get the nodes that have not suffered any alarms in the last weeks:
eboxes_nodes = nodes.loc[nodes["type"] == "box"]
non_error_eboxes_nodes = eboxes_nodes.loc[~eboxes_nodes["id"].isin(eboxes_alarms["id"])]

# Now we have to build a dataframe with the same structure as light_eboxes for this non_error_eboxes:
non_error_eboxes = pd.DataFrame(
    {
        "id": non_error_eboxes_nodes["id"],
        "week-4": eboxes_alarms["week-4"][0],
        "hours_week-4": 0,
        "week-3": eboxes_alarms["week-3"][0],
        "hours_week-3": 0,
        "week-2": eboxes_alarms["week-2"][0],
        "hours_week-2": 0,
        "week-1": eboxes_alarms["week-1"][0],
        "hours_week-1": 0,
        "current_week": eboxes_alarms["current_week"][0],
        "hours_current_week": 0,
        "lat": non_error_eboxes_nodes["lat"],
        "lon": non_error_eboxes_nodes["lon"]
    }
)
# Create the dataframe ready for the model:

non_error_eboxes = join_eboxes_alarms_readings_meteo(
    eboxes_alarms = non_error_eboxes,
    eboxes_powerReactivePeak = powerReactivePeak,
    eboxes_powerReactive = powerReactive,
    eboxes_powerActive = powerActive,
    eboxes_powerActivePeak = powerActivePeak,
    meteo = meteo
)

# Now for the the lights:

light_alarms = first_lights_preprocess(light_alarms, for_predicting=True)

light_alarms = big_preprocess_lights(
    light_alarms = light_alarms,
    for_predicting = True,
    predicting_min_date = predicting_min,
    predicting_max_date = predicting_max
)

light_alarms = pd.merge(light_alarms, nodes, on="id", how="left")

light_alarms = join_light_alarms_readings_meteo(
    light_errors = light_alarms,
    eboxes_powerReactivePeak = powerReactivePeak,
    eboxes_powerReactive = powerReactive,
    eboxes_powerActive = powerActive,
    eboxes_powerActivePeak = powerActivePeak,
    meteo = meteo
)

# Untill this point we have just considered the lights that have had an error in the last 4 weeks. 
# We should consider too all the other alarms that have no errors in the last 4 weeks and put them
# in a dataframe format identical to the light-alarms dataframe. This way we will be able to do predictionns
# for all the lights in the city.

# Let's get the nodes that have not suffered any alarms in the last weeks:
light_nodes = nodes.loc[nodes["type"] == "light"]
non_error_lights_nodes = light_nodes.loc[~light_nodes["id"].isin(light_alarms["id"])]

# Now we have to build a dataframe with the same structure as light_alarms for this non_error_lights:
non_error_lights = pd.DataFrame(
    {
        "id": non_error_lights_nodes["id"],
        "week-4": light_alarms["week-4"][0],
        "hours_week-4": 0,
        "week-3": light_alarms["week-3"][0],
        "hours_week-3": 0,
        "week-2": light_alarms["week-2"][0],
        "hours_week-2": 0,
        "week-1": light_alarms["week-1"][0],
        "hours_week-1": 0,
        "current_week": light_alarms["current_week"][0],
        "hours_current_week": 0,
        "ebox_id": non_error_lights_nodes["ebox_id"],
        "lat": non_error_lights_nodes["lat"],
        "lon": non_error_lights_nodes["lon"]
    }
)
# Create the dataframe ready for the model:

non_error_lights = join_light_alarms_readings_meteo(
    light_errors = non_error_lights,
    eboxes_powerReactivePeak = powerReactivePeak,
    eboxes_powerReactive = powerReactive,
    eboxes_powerActive = powerActive,
    eboxes_powerActivePeak = powerActivePeak,
    meteo = meteo
)

AA0000011277
Execution time:0.03579998016357422
AA000000AC39
AA000000AC94
AA000000AF94
AA000000B302
AA000000B328
AA000000B366
AA000000B367
AA000000B36E
AA000000B719
AA000000B8A6
AA000000BD3C
AA000000BE7F
AA00000148E5
AA0000017347
AA000001734B
AA0000018F1C
AA00000192D8
AA00000195F9
Execution time:1.709451675415039


In [16]:
print(predicting_min)
print(predicting_max)

2023-04-17
2023-05-21


In [22]:
dates_range = pd.date_range(start=predicting_min, end=predicting_max, freq="W")
dates_range = [str(return_monday(elem)) for elem in dates_range]
dates_range

['2023-04-17', '2023-04-24', '2023-05-01', '2023-05-08', '2023-05-15']

In [23]:
light_alarms

,id,week-4,hours_week-4,week-3,hours_week-3,week-2,hours_week-2,week-1,hours_week-1,current_week,...,Wind_max_max_current_week,Wind_avg_avg_current_week,Wind_avg_std_current_week,Wind_min_min_current_week,Pres_max_max_current_week,Pres_avg_avg_current_week,Pres_avg_std_current_week,Pres_min_min_current_week,Precipitation_avg_current_week,Precipitation_sum_current_week
0,AA000000AC39,2023-04-17,0.000000,2023-04-24,0.035278,2023-05-01,0.000000,2023-05-08,0.000000,2023-05-15,...,20,5.742857,1.632847,0,28.1,27.828571,0.149603,27.6,0.0,0.0
1,AA000000AC94,2023-04-17,0.000000,2023-04-24,0.000000,2023-05-01,0.003611,2023-05-08,0.013611,2023-05-15,...,20,5.742857,1.632847,0,28.1,27.828571,0.149603,27.6,0.0,0.0
2,AA000000AF94,2023-04-17,0.019722,2023-04-24,0.000000,2023-05-01,0.000000,2023-05-08,0.013889,2023-05-15,...,20,5.742857,1.632847,0,28.1,27.828571,0.149603,27.6,0.0,0.0
3,AA000000B302,2023-04-17,13.598889,2023-04-24,0.000000,2023-05-01,0.020278,2023-05-08,0.024722,2023-05-15,...,20,5.742857,1.632847,0,28.1,27.828571,0.149603,27.6,0.0,0.0
4,AA000000B328,2023-04-17,0.000000,2023-04-24,0.820000,2023-05-01,0.000000,2023-05-08,0.000000,2023-05-15,...,20,5.742857,1.632847,0,28.1,27.828571,0.149603,27.6,0.0,0.0
5,AA000000B366,2023-04-17,0.000000,2023-04-24,8.363333,2023-05-01,0.000000,2023-05-08,0.000000,2023-05-15,...,20,5.742857,1.632847,0,28.1,27.828571,0.149603,27.6,0.0,0.0
6,AA000000B367,2023-04-17,0.017222,2023-04-24,27.837500,2023-05-01,14.141111,2023-05-08,14.355833,2023-05-15,...,20,5.742857,1.632847,0,28.1,27.828571,0.149603,27.6,0.0,0.0
7,AA000000B36E,2023-04-17,54.708611,2023-04-24,69.746111,2023-05-01,71.045556,2023-05-08,72.341667,2023-05-15,...,20,5.742857,1.632847,0,28.1,27.828571,0.149603,27.6,0.0,0.0
8,AA000000B719,2023-04-17,0.104722,2023-04-24,0.419167,2023-05-01,0.025556,2023-05-08,0.213889,2023-05-15,...,20,5.742857,1.632847,0,28.1,27.828571,0.149603,27.6,0.0,0.0
9,AA000000B8A6,2023-04-17,0.000000,2023-04-24,4.895278,2023-05-01,0.000000,2023-05-08,0.000000,2023-05-15,...,20,5.742857,1.632847,0,28.1,27.828571,0.149603,27.6,0.0,0.0


In [11]:
def make_predictions_lights(light_alarms: pd.DataFrame, model_type: str = "default") -> None:

    # In this case we don't use the ada boost combined predictor and the model will take into accout the readings. This
    # model has an overall better accuracy than the adboc but fails to detect the sudden errors
    if model_type == "default":
        with open("predictive_models/ada_model_readings.pk1", "rb") as file:
            ada_model = pickle.load(file)
        with open("predictive_models/ada_prob_readings.pk1", "rb") as file:
            prob_ada_model = pickle.load(file)["prob_ada_model"]
        
        df = light_alarms.copy()

        # Drop some usless columns for the model:
        drop_cols = [
            col for col in df.columns if
                (col in ["lat", "lon"]) | 
                (col == "Unnamed: 0") |
                (col.startswith("week")) |
                (col == "current_week") |
                (col == "type") |
                (col == "ebox_id") |
                (col == "location")
        ]
        df = df.drop(drop_cols, axis=1)

        # Interpolate some left missing values:
        df = df.fillna(df.mean(numeric_only=True))

        predictions = ada_model.predict_proba(df.drop("id", axis=1))[:, 1]

        predictions_out = pd.DataFrame(
            {
                "id": df["id"],
                "pred": predictions
            }
        )

        print("Predictions:")
        print("Probability threshold recommended for this model for lights: " + str(prob_ada_model))
        print(predictions_out)

    # In this case we will use the adboc model. This model has a worse overall accuracy than the default but has a better chance 
    # at dettecting sudden errors. The model does not use the readings in this case.
    if model_type == "adboc":
        with open("predictive_models/ada_model.pk1", "rb") as file:
            ada_model = pickle.load(file)
        with open("predictive_models/ada_sudden_model.pk1", "rb") as file:
            ada_sudden_model = pickle.load(file)
        with open("predictive_models/ada_prob.pk1", "rb") as file:
            probs_dict = pickle.load(file)
            prob_ada_model = probs_dict["prob_ada_model"]
            prob_sudden_model = probs_dict["prob_sudden_model"]
        
        df = light_alarms.copy()
        drop_cols = [
                    col for col in df.columns if 
                    (col.startswith("power")) | (col.startswith("Active")) | (col.startswith("Reactive") | 
                    (col in ["lat", "lon"])) | 
                    (col == "Unnamed: 0") |
                    (col.startswith("week")) |
                    (col == "current_week") |
                    (col == "type") |
                    (col == "ebox_id") |
                    (col == "location")
                ]
        df = df.drop(drop_cols, axis=1)
        df = df.fillna(df.mean(numeric_only=True))

        predictions = adboc_predict(
            df = df,
            ada_model = ada_model,
            ada_sudden_model = ada_sudden_model,
            prob_threshold_ada_model = prob_ada_model,
            prob_threshold_ada_sudden_model = prob_sudden_model,
        )

        predictions_out = pd.DataFrame(
            {
                "id": predictions.keys(),
                "pred": predictions.values()
            }
        )

        print("Predictions:")
        print("Probability threshold recommended for the model ada_model for lights: " + str(prob_ada_model))
        print("Probability threshold recommended for the model ada_sudden_model for lights: " + str(prob_sudden_model))
        print(predictions_out)

def make_predictions_eboxes(eboxes_alarms: pd.DataFrame) -> None:

    with open("predictive_models/ada_model_eboxes.pk1", "rb") as file:
        ada_model = pickle.load(file)

    with open("predictive_models/ada_prob_eboxes.pk1", "rb") as file:
        probs_dict = pickle.load(file)
        prob_ada_model = probs_dict["prob_ada_model"]
    
    df = eboxes_alarms.copy()
    drop_cols = [
                col for col in df.columns if 
                (col.startswith("power")) | (col.startswith("Active")) | (col.startswith("Reactive") | 
                (col in ["lat", "lon"])) | 
                (col == "Unnamed: 0") |
                (col.startswith("week")) |
                (col == "current_week") |
                (col == "type") |
                (col == "location")
            ]
    df = df.drop(drop_cols, axis=1)
    df = df.fillna(df.mean(numeric_only=True))

    predictions = ada_model.predict_proba(df.drop("id", axis=1))[:, 1]

    predictions_out = pd.DataFrame(
        {
            "id": df["id"],
            "pred": predictions
        }
    )

    print("Predictions:")
    print("Probability threshold recommended for this model for eboxes: " + str(prob_ada_model))
    print(predictions_out)

NameError: name 'light_alarms' is not defined

# Predictions test for eboxes:

In [67]:
def first_eboxes_preprocess(eboxes_alarms: pd.DataFrame, for_predicting = False, min_date = "1982-01-04", max_date = "2023-04-04") -> pd.DataFrame:
    """
    Performs basic preprocessing on the raw data of alarms such as filtering the errors that
    we want to predict ("lightcomm", "lighterr"), remove useless columns, parse dates and 
    eliminate dates out of a defined range.
    """

    eboxes_alarms = eboxes_alarms.loc[eboxes_alarms["subtype"] == "mcbpower"]

    eboxes_alarms = eboxes_alarms[["id", "dated", "subtype", "flag"]]

    eboxes_alarms["dated"] = pd.to_datetime(eboxes_alarms["dated"])

    if not for_predicting:
        # We have to do some filtering for the raw data because in the datasets for training we have some nonsense data
        # like data from the future.
        eboxes_alarms = eboxes_alarms.loc[(min_date <= eboxes_alarms["dated"]) & (eboxes_alarms["dated"] <= max_date)]

    return eboxes_alarms

def big_preprocess_eboxes(eboxes_alarms: pd.DataFrame, predicting_min_date: str = None, predicting_max_date: str = None, for_predicting: bool = False) -> pd.DataFrame:
    """
    The function converts the data of the alarms into a dataframe that is usable for training the models.
    Review the coments on the code to understand step by step what the code does.
    """
    
    start_time = time.time()

    # We create a dataframe of each one of the ids and we will clean the alarms. The final objective is to calculate
    # the percentage of time the ebox is not working wich is the time that passes from an alarm "off" utill it is 
    # turned "on". In the dataframe we have cases in wich there are two consecutive "on" or "off" alarms so we have
    # filter and just keep the first "on" and the first "off" in this case so we get the real time the ebox has been
    # not functioning. In the case that the alarm is turned "on" during a week and it is not turned "off" untill the next
    # week we will insert a fake "off" alarm at the last moment of the week and we will turn it "on" again in the first moment
    # of the following week.

    # The first step will be to dowload the data:
    # Read recipe inputs
    df = eboxes_alarms.copy()

    # We will reuse the code of the light alarms for the eboxes: The main thing that we have to modify for the eboxes
    # is that for the alarm subtype brdpower the flag "off" means that the ebox is suffering a breakdown and untill the
    # alarm turns "on" we will consider that the ebox has been down. This is exactly opposite to what happened with the
    # lamposts.

    # The first step is to change the name of the column subtype. Even tough this columns means a subtype of alarm, in this
    # code and from now on we will rename it as alarm so we are able to replicate a similar code as the one used for the lights
    df.rename(columns={"subtype": "alarm"}, inplace=True)

    # There are four different flags in this case: {off, on, offM, onM} depending if the error has been detected digitaly or manualy.
    # we do not care so we fill change this categories to {on off}:
    df["flag"] = df["flag"].replace(["onM"], "on")
    df["flag"] = df["flag"].replace(["offM"], "off")

    # As we already said the flags in the eboxes are opposite as the one in the lamposts, so to be able to use the same
    # code we will just change the values "off" to "on" and the "on" to "off" and then apply the same exact code that
    # we used for the lamposts.
    df["flag"] = df["flag"].replace({'on': 'offf', 'off': 'on'}).replace({"offf": "off"})

    # Sort by id and date of the alarm:
    df["dated"] = pd.to_datetime(df["dated"])
    df = df.sort_values(["id", "dated"])

    # Generate all the weeks from the first day we have data untill the last day. We will need this weeks later to
    # do a left join with the weeks that there are alarms

    # If we are preprocessing the data for doing predictions we will want to add the date_min and date max manualy,
    # otherwise the function will only consider the dates of the errors and you may end up with nan rows
    if for_predicting: 
        start_date = pd.to_datetime(predicting_min_date)
        end_date = pd.to_datetime(predicting_max_date)
    else:
        start_date = df["dated"].min()
        end_date = return_monday(df["dated"].max()) + pd.Timedelta(days=7)

    weeks = pd.date_range(start=start_date, end=end_date, freq='W').floor("D").strftime('%Y-%m-%d %H:%M:%S').tolist()
    # Transform to a dataframe to do operations later:
    weeks = pd.DataFrame(weeks, columns=["week"])
    weeks["week"] = pd.to_datetime(weeks["week"])
    weeks["week"] = weeks["week"].apply(lambda x: return_monday(x))

    # For each unique id we will implement all the code:
    # First create the dataframe where we will store all the resturned data:
    general_lag_dataframe = pd.DataFrame()

    # List of all the ids:
    ids_list = df["id"].unique()

    # Store the name of the columns in oder to later reordenate the final returned dataframe.
    if for_predicting:
        columns_order = ["id", "week-4", "hours_week-4", "week-3", "hours_week-3", "week-2", "hours_week-2", "week-1", "hours_week-1",
                        "current_week", "hours_current_week"]
    else:     
        columns_order = ["id", "week-4", "hours_week-4", "week-3", "hours_week-3", "week-2", "hours_week-2", "week-1", "hours_week-1",
                        "current_week", "hours_current_week", "week+1", "hours_week+1", "week+2", "hours_week+2", "week+3", "hours_week+3", "week+4", "hours_week+4"]

    # Generate a dataframe for each one of the ids and then apply all the transformations. Once done, add the data
    # to the dataframe general_lag_dataframe.
    for idd in ids_list:
        print(idd)
        
        # Dataframe of all the alarms for the id "idd":
        tt = df.loc[df["id"] == idd]
        
        #For each one of the elements in the list we have: elem[0] contains the week represented by sunday 
        # and elem[1] has the data in a dataframe
        grouped_weeks = list(tt.groupby(pd.Grouper(key="dated", freq="W")))
        # Here we will transform all the tuples to lists so we can change the elements after
        grouped_weeks = [list(elem) for elem in grouped_weeks]

        # Now we have to filtrate the empty dataframes that generates groupby for the weeks where there are no alarms
        grouped_weeks = [elem for elem in grouped_weeks if not elem[1].empty]

        # Eliminate the repeated flags such as "off" followed by "off" or "on" followed by "on" for the first element
        # of the list grouped_weeks. We do it because in the iteration we will not consider it so we have to do it now
        first_week_data = grouped_weeks[0][1]
        first_week_data["prev_flag"] = first_week_data["flag"].shift()

        first_week_data = first_week_data.loc[
            ((first_week_data["flag"] == "on") & (first_week_data["prev_flag"] == "off")) |
            ((first_week_data["flag"] == "off") & (first_week_data["prev_flag"] == "on")) |
            ((first_week_data["flag"] == "off") & (first_week_data["prev_flag"].isna())) | # This two last comparisons are for the first rows because they don't have a previous flag
            ((first_week_data["flag"] == "on") & (first_week_data["prev_flag"].isna()))
        ]

        # We store it to the data of the first week:
        grouped_weeks[0][1] = first_week_data

        # Here we begin the iteration for all the other weeks:
        for i, (week, data) in enumerate(grouped_weeks[1:], 1): # We do not consider the first week because it has no previous week to check
            # Get the data and week from the previous week:
            previous_week, previous_week_data = grouped_weeks[i-1][0], grouped_weeks[i-1][1]

            # Eliminate the repeated flags such as "off" followed by "off" or "on" followed by "on"
            data["prev_flag"] = data["flag"].shift()
            data = data.loc[
                ((data["flag"] == "on") & (data["prev_flag"] == "off")) |
                ((data["flag"] == "off") & (data["prev_flag"] == "on")) |
                ((data["flag"] == "off") & (data["prev_flag"].isna())) | # This two last comparisons are for the first rows because they don't have a previous flag
                ((data["flag"] == "on") & (data["prev_flag"].isna()))
            ]

            # Data that we will need:
            last_moment_previous_week = previous_week + pd.Timedelta(hours=23, minutes=59, seconds=59) #We will need the last moment of the week
            first_moment_current_week = (week - pd.Timedelta(days=week.dayofweek)).replace(hour=0, minute=0, second=0) #We will need to the first moment of the current week

            #We have to get the last flag of the previous week
            last_flag_previous_week = previous_week_data.loc[previous_week_data.index[-1], "flag"]

            if last_flag_previous_week == "on":
                # If the last flag from the previous week is "on" then we have to set a new row on the previous week data
                # in the last position to set a flag "off". Then, in the current week data we will add a new row before the
                # first week to set again the alarm to "on"

                # Here we create the new row to set the alarm "off" in the previous week
                new_row_previous_week = pd.DataFrame(
                    {
                    "id": [idd],
                    "dated": [last_moment_previous_week],
                    "alarm": ["turn_off_end_week"], # We will put this in alarm to know witch alarms where inserted by us
                    "flag": ["off"],
                    "prev_flag": ["Empty"] # Empty because we do not need it anymore and it will help us identify this rows
                    }
                )

                # We create the new df with the new row at the very end of the week
                new_previous_week_data = pd.concat(
                    [previous_week_data, new_row_previous_week],
                    sort=True # Remove the warning of pd.concat
                )
                # We update the dataframe in the list grouped_weeks
                grouped_weeks[i-1][1] = new_previous_week_data

                # Now we have to set the flag "on" in the first moment of the current week:
                # Here we create the new row to set the alarm "on" in the current week
                new_row_current_week = pd.DataFrame(
                    {
                    "id": [idd],
                    "dated": [first_moment_current_week],
                    "alarm": ["turn_on_begining_week"], # We will put this in alarm to know witch alarms where inserted by us
                    "flag": ["on"],
                    "prev_flag": ["Empty"] # Empty because we do not need it anymore and it will help us identify this rows
                    }
                )

                # We create the new df with the new row at the very begining of the week
                new_current_week_data = pd.concat(
                    [new_row_current_week, data],
                    sort=True # Remove the warning of pd.concat
                )
                # We update the dataframe in the list grouped_weeks
                grouped_weeks[i][1] = new_current_week_data
            else:
                # Simply update the dataframe with the same but with removed rows that contain two identical flags in a row
                grouped_weeks[i][1] = data

        # Once this is done we have to check if the last alarm of the last week is "on". In this case we will add a row turning it off
        # in the last moment of the week:
        last_recorded_week, last_recorded_week_data = grouped_weeks[-1][0], grouped_weeks[-1][1]

        # Get the last flag from the last week
        last_flag = last_recorded_week_data["flag"].values[-1]
        # and get the last moment of the last week
        last_moment_last_recorded_week = last_recorded_week + pd.Timedelta(hours=23, minutes=59, seconds=59)

        if last_flag == "on":
            new_row_last_flag = pd.DataFrame(
                {
                "id": [idd],
                "dated": [last_moment_last_recorded_week],
                "alarm": ["turn_off_end_last_week"], # We will put this in alarm to know witch alarms where inserted by us
                "flag": ["off"],
                "prev_flag": ["Empty"] # Empty because we do not need it anymore and it will help us identify this rows
                }
            )

            last_recorded_week_data = pd.concat(
                [last_recorded_week_data, new_row_last_flag],
                sort=True
            )

            grouped_weeks[-1][1] = last_recorded_week_data

        # Now we have to concat all the dataframes from all the weeks contained in the list grouped_weeks in one big dataframe
        concatenated_weeks = pd.concat(
            [week_data[1] for week_data in grouped_weeks],
            sort=True # Remove the warning of pd.concat
        )

        # At this point there are some cases where we will still have two "on" alarms or two "off" alarms in a row. For example
        # in the case that we have two weeks in a row where we only have "on" alarms utill now the code is going to return the 
        # begining of the end of the previous week with "off", the beggining of the current week with "off" and before the "off"
        # of the end of the week we will still have an "on" alarm. This is caused because the deletion of the same alarms in a row is done 
        # before the add of the new rows in the beggining and end of the week

        # So let's eliminate this cases too:
        # first we have to frop the old prev_flag column that now is useless:
        concatenated_weeks.drop("prev_flag", axis=1, inplace=True)

        # and create the new one:
        concatenated_weeks["prev_flag_concat"] = concatenated_weeks["flag"].shift()

        concatenated_weeks = concatenated_weeks.loc[
            ((concatenated_weeks["flag"] == "on") & (concatenated_weeks["prev_flag_concat"] == "off")) |
            ((concatenated_weeks["flag"] == "off") & (concatenated_weeks["prev_flag_concat"] == "on")) |
            ((concatenated_weeks["flag"] == "off") & (concatenated_weeks["prev_flag_concat"].isna())) | # This two last comparisons are for the first rows because they don't have a previous flag
            ((concatenated_weeks["flag"] == "on") & (concatenated_weeks["prev_flag_concat"].isna()))
        ]

        # We have some weeks where there is no data and thus if this weeks are between an "on" and "off" flag they will not
        # appear on the dataframe. What we will do is a left join with the variable week generated at the begining
        # of the notebook to see the rows that do not appear. Then we will activate the alarm at the begining of the week
        # and deactivate it at the end of the same week:

        # Set the day to monday
        concatenated_weeks["week"] = concatenated_weeks["dated"].apply(lambda x: return_monday(x))
        # Left join with weeks to detect the missing weeks
        concatenated_weeks_merged = pd.merge(weeks, concatenated_weeks, on="week", how="left")

        # Store the result of the filling:
        filled_dataframe = pd.DataFrame()

        # We add the first row
        if not pd.isna(concatenated_weeks_merged.iloc[0]["alarm"]):
            first_new_row = pd.DataFrame(
                {
                    "week": [concatenated_weeks_merged.iloc[0]["week"]],
                    "alarm": [concatenated_weeks_merged.iloc[0]["alarm"]],
                    "dated": [concatenated_weeks_merged.iloc[0]["dated"]],
                    "flag": [concatenated_weeks_merged.iloc[0]["flag"]],
                    "id": [concatenated_weeks_merged.iloc[0]["id"]]
                }
            )

            filled_dataframe = pd.concat(
                [filled_dataframe, first_new_row],
                sort=True
            )

        # Begin the iteration where we will add one by one the rows to the dataframe filled_dataframe
        for i in range(1, len(concatenated_weeks_merged)-1):

            current_row = concatenated_weeks_merged.iloc[i]

            # If we find a normal row we add it to the dataframe
            if (current_row["alarm"] == "mcbpower") | (current_row["alarm"] == "turn_off_end_week") | (current_row["alarm"] == "turn_on_begining_week"):
                current_row["dated"] = pd.to_datetime(current_row["dated"])

                new_row = pd.DataFrame(
                    {
                        "week": [current_row["week"]],
                        "alarm": [current_row["alarm"]],
                        "dated": [current_row["dated"]],
                        "flag": [current_row["flag"]],
                        "id": [current_row["id"]]
                    }
                )
                filled_dataframe = pd.concat([filled_dataframe, new_row], sort=True)

            if not filled_dataframe.empty:
                # If the last row of the filled dataframe is a "turn_off_week" ant the current is a Emty we add the "on" and
                # off for the begining and the end of the week
                last_row_filled_dataframe = filled_dataframe.iloc[-1]

                if (pd.isna(current_row["alarm"])) & (last_row_filled_dataframe["alarm"] in ["turn_off_end_week", "turn_off_end_week_filled"]):

                    new_row_begining_dated = datetime.datetime.combine(current_row["week"], datetime.time(0,0,0))
                    new_row_begining_week = pd.DataFrame(
                        {
                            "week": [current_row["week"]],
                            "alarm": ["turn_on_begining_week_filled"],
                            "dated": [new_row_begining_dated], # Add the time to the date
                            "flag": ["on"],
                            "id": [idd]
                        }
                    )
                    # We have to add 6 days because the representative of the week in this case is 
                    new_row_end_dated = datetime.datetime.combine(current_row["week"]+ pd.Timedelta(days=6), datetime.time(23,59,59))
                    new_row_end_week = pd.DataFrame(
                        {
                            "week": [current_row["week"]],
                            "alarm": ["turn_off_end_week_filled"],
                            "dated": [new_row_end_dated], # In this case we add the time to represent the last moment of the week
                            "flag": ["off"],
                            "id": [idd]
                        }
                    )

                    filled_dataframe = pd.concat(
                        [filled_dataframe, new_row_begining_week, new_row_end_week],
                        sort=True
                    )

        # Again we have to do a left join with the dataframe weeks to detect the NaN values:
        filled_dataframe_merged = pd.merge(weeks, filled_dataframe, on="week", how="left")
        
        # Now for each one of the weeks we have to calculate the total time passed between an "on" alarm and an "off" alarm

        # In this dataframe we will store the amount of hours of a ebox that has been malfunctioning for each week
        week_hours_dataframe = pd.DataFrame()
        for week in weeks["week"]:
            # In this variable we will store the amount of hours for this week:
            total_hours = 0
            on_timestamp = None

            # Dataframe with the alarms of the week:
            week_alarms_dataframe = filled_dataframe_merged.loc[filled_dataframe_merged["week"] == week]
            # Iterate trough the df to count the hours:
            for _, row in week_alarms_dataframe.iterrows():
                if row["flag"] == "on":
                    on_timestamp = row["dated"]

                if (row["flag"] == "off") & (on_timestamp is not None):
                    total_hours += (row["dated"] - on_timestamp).total_seconds() / 3600

                    on_timestamp = None

            new_week_hours = pd.DataFrame(
                {
                    "id": [idd],
                    "week": [week],
                    "malfunctioning_hours": [total_hours] 
                }
            )

            week_hours_dataframe = pd.concat(
                [week_hours_dataframe, new_week_hours],
                sort=True
            )

        # Now we want to get the data in the format:
        # row = {"week-4": date_week_prev_4, "hours_week-4": hours_week_prev_4, ..., "week-1": date_week_prev_1, "hours_week-1": hours_week_prev_1, "current_week": date_current_week, "hours_current_week":  "week+1": date_week_next_1, "hours_week+1": hours_week_next_1 ..., "week+4": date_week_next_4, "hours_week+4": hours_week_next_4}

        # In this dataframe we will store the data in the format we have mentioned:
        lag_dataframe = pd.DataFrame()

        # Here we have to follow two different paths. The first one will be when preparing the data for training and the other one will be when 
        # preparing the data for predicting. We shall begin with predicting:

        if for_predicting:
            lag_dataframe = pd.DataFrame(
                {
                    "id": [idd],

                    "week-4": [week_hours_dataframe.iloc[0]["week"]],
                    "hours_week-4": [week_hours_dataframe.iloc[0]["malfunctioning_hours"]],

                    "week-3": [week_hours_dataframe.iloc[1]["week"]],
                    "hours_week-3": [week_hours_dataframe.iloc[1]["malfunctioning_hours"]],

                    "week-2": [week_hours_dataframe.iloc[2]["week"]],
                    "hours_week-2": [week_hours_dataframe.iloc[2]["malfunctioning_hours"]],

                    "week-1": [week_hours_dataframe.iloc[3]["week"]],
                    "hours_week-1": [week_hours_dataframe.iloc[3]["malfunctioning_hours"]],

                    "current_week": [week_hours_dataframe.iloc[4]["week"]],
                    "hours_current_week": [week_hours_dataframe.iloc[4]["malfunctioning_hours"]]
                }
            )

            # Reorder the dataframe so it is in the same order we have defined:
            lag_dataframe = lag_dataframe[columns_order]
            
            # Add the dataframe to the general one:
            general_lag_dataframe = pd.concat(
                [general_lag_dataframe, lag_dataframe],
                sort=True
            )

        else:
            # Begin the loop at 4 and end at -4 so we don't get the error: "Out of range"
            for i in range(4, len(weeks)-4):
                # Create the new row to add:
                to_add_row = pd.DataFrame(
                    {
                        "id": [idd],

                        "week-4": [week_hours_dataframe.iloc[i-4]["week"]],
                        "hours_week-4": [week_hours_dataframe.iloc[i-4]["malfunctioning_hours"]],

                        "week-3": [week_hours_dataframe.iloc[i-3]["week"]],
                        "hours_week-3": [week_hours_dataframe.iloc[i-3]["malfunctioning_hours"]],

                        "week-2": [week_hours_dataframe.iloc[i-2]["week"]],
                        "hours_week-2": [week_hours_dataframe.iloc[i-2]["malfunctioning_hours"]],

                        "week-1": [week_hours_dataframe.iloc[i-1]["week"]],
                        "hours_week-1": [week_hours_dataframe.iloc[i-1]["malfunctioning_hours"]],

                        "current_week": [week_hours_dataframe.iloc[i]["week"]],
                        "hours_current_week": [week_hours_dataframe.iloc[i]["malfunctioning_hours"]],

                        "week+1": [week_hours_dataframe.iloc[i+1]["week"]],
                        "hours_week+1": [week_hours_dataframe.iloc[i+1]["malfunctioning_hours"]],

                        "week+2": [week_hours_dataframe.iloc[i+2]["week"]],
                        "hours_week+2": [week_hours_dataframe.iloc[i+2]["malfunctioning_hours"]],

                        "week+3": [week_hours_dataframe.iloc[i+3]["week"]],
                        "hours_week+3": [week_hours_dataframe.iloc[i+3]["malfunctioning_hours"]],

                        "week+4": [week_hours_dataframe.iloc[i+4]["week"]],
                        "hours_week+4": [week_hours_dataframe.iloc[i+4]["malfunctioning_hours"]]
                    }
                )

                lag_dataframe = pd.concat(
                    [lag_dataframe, to_add_row],
                    sort=True,
                    ignore_index=True
                )

            # Reorder the dataframe so it is in the same order we have defined:
            lag_dataframe = lag_dataframe[columns_order]
            
            # Add the dataframe to the general one:
            general_lag_dataframe = pd.concat(
                [general_lag_dataframe, lag_dataframe],
                sort=True
            )
        
    # Reordenate with the list columns_order:
    general_lag_dataframe = general_lag_dataframe[columns_order]

    end_time = time.time()

    print("Execution time:" + str(end_time - start_time))

    return general_lag_dataframe

def change_format(date: str) -> str:
    """
    Get the date from a datetime object and convert it into a string
    """
    return str(date)[:10]

def join_eboxes_alarms_readings_meteo(
        eboxes_alarms: pd.DataFrame, 
        eboxes_powerReactivePeak: pd.DataFrame,
        eboxes_powerReactive: pd.DataFrame,
        eboxes_powerActive: pd.DataFrame,
        eboxes_powerActivePeak: pd.DataFrame,
        meteo: pd.DataFrame
    ) -> pd.DataFrame:
    """
    Joins the preprocessed data from light_alarms, readings, and meteo data by weeks
    """

    #Change meteo date format
    meteo['dated'] = meteo['dated'].apply(change_format)
    for week in ["week-4", "week-3", "week-2", "week-1", "current_week"]:
        eboxes_alarms[week] = eboxes_alarms[week].apply(lambda x: str(x))

    display(eboxes_alarms)
    display(eboxes_powerReactivePeak)
    # joins for the dataset eboxes_powerReactivePeak:
    for week in ["week-4", "week-3", "week-2", "week-1", "current_week"]:

        eboxes_alarms = pd.merge(
            eboxes_alarms, 
            eboxes_powerReactivePeak.rename(columns={"dated": week}), 
            on=["id", week],
            how="left"
        )
        
        eboxes_alarms.rename(columns={"ReactivePeak": "ReactivePeak_" + week}, inplace=True)

    print("Here!")
    # joins for the dataset eboxes_powerReactive:
    for week in ["week-4", "week-3", "week-2", "week-1", "current_week"]:

        eboxes_alarms = pd.merge(
            eboxes_alarms, 
            eboxes_powerReactive.rename(columns={"dated": week}), 
            on=["id", week],
            how="left"
        )
        
        eboxes_alarms.rename(
            columns={
                "powerReactive_sum": "powerReactive_sum_" + week,
                "powerReactive_p1": "powerReactive_p1_" + week,
                "powerReactive_p2": "powerReactive_p2_" + week,
                "powerReactive_p3": "powerReactive_p3_" + week
                    },
            inplace=True)
    
    print("here!")

    # joins for the dataset eboxes_powerActivePeak:
    for week in ["week-4", "week-3", "week-2", "week-1", "current_week"]:

        eboxes_alarms = pd.merge(
            eboxes_alarms, 
            eboxes_powerActivePeak.rename(columns={"dated": week}), 
            on=["id", week],
            how="left"
        )
        
        eboxes_alarms.rename(columns={"ActivePeak": "ActivePeak_" + week}, inplace=True)

    # joins for the dataset eboxes_powerActive:
    for week in ["week-4", "week-3", "week-2", "week-1", "current_week"]:

        eboxes_alarms = pd.merge(
            eboxes_alarms, 
            eboxes_powerActive.rename(columns={"dated": week}), 
            on=["id", week],
            how="left"
        )
        
        eboxes_alarms.rename(
            columns={
                "powerActive_sum": "powerActive_sum_" + week,
                "powerActive_p1": "powerActive_p1_" + week,
                "powerActive_p2": "powerActive_p2_" + week,
                "powerActive_p3": "powerActive_p3_" + week
                    },
            inplace=True)
        

    # joins for the dataset meteo_canyelles:
    for week in ["week-4", "week-3", "week-2", "week-1", "current_week"]:

        eboxes_alarms = pd.merge(
            eboxes_alarms, 
            meteo.rename(columns={"dated": week}), 
            on=[week],
            how="left"
        )
        
        eboxes_alarms.rename(
            columns={
                "Dew_max_max": "Dew_max_max_" + week,
                "Hum_min_min": "Hum_min_min_" + week,
                "Wind_max_max": "Wind_max_max_" + week,
                "Temp_max_max": "Temp_max_max_" + week,
                "Temp_min_min": "Temp_min_min_" + week,
                "Pres_min_min": "Pres_min_min_" + week,
                "Wind_avg_mean": "Wind_avg_avg_" + week,
                "Wind_avg_std": "Wind_avg_std_" + week,
                "Hum_avg_mean": "Hum_avg_avg_" + week,
                "Dew_avg_mean": "Dew_avg_avg_" + week,
                "Dew_avg_std": "Dew_avg_std_" + week,
                "Hum_max_max": "Hum_max_max_" + week,
                "Temp_avg_mean": "Temp_avg_avg_" + week,
                "Temp_avg_std": "Temp_avg_std_" + week,
                "Pres_max_max": "Pres_max_max_" + week,
                "Pres_avg_mean": "Pres_avg_avg_" + week,
                "Pres_avg_std": "Pres_avg_std_" + week,
                "Precipitation_mean": "Precipitation_avg_" + week,
                "Precipitation_sum": "Precipitation_sum_" + week,
                "Dew_min_min": "Dew_min_min_" + week,
                "Wind_min_min": "Wind_min_min_" + week,
                "Hum_avg_std": "Hum_avg_std_" + week
                    },
            inplace=True)
    
    return eboxes_alarms

In [68]:
eboxes_alarms = pd.read_csv("/home/leibniz/Desktop/IHMAN/data_to_predict/eboxes_alarms.csv")

AA0000011277
Execution time:0.04645967483520508


,id,week-4,hours_week-4,week-3,hours_week-3,week-2,hours_week-2,week-1,hours_week-1,current_week,hours_current_week
0,AA0000011277,2023-04-17,0.0,2023-04-24,0.0,2023-05-01,1.061111,2023-05-08,0.815278,2023-05-15,0.0


,ebox_id,dated,ReactivePeak
0,AA000000AA88,2023-04-17,1.577872e+06
1,AA000000AA88,2023-04-24,1.498229e+06
2,AA000000AA88,2023-05-01,1.438945e+06
3,AA000000AA88,2023-05-08,1.413214e+06
4,AA000000AA88,2023-05-15,1.378143e+06
5,AA000000D458,2023-04-17,2.168676e+06
6,AA000000D458,2023-04-24,2.099196e+06
7,AA000000D458,2023-05-01,2.045789e+06
8,AA000000D458,2023-05-08,2.040446e+06
9,AA000000D458,2023-05-15,2.023542e+06


KeyError: 'id'

In [3]:
ee = len(pd.DataFrame()) == 0

In [4]:
ee

True

In [6]:
if not True:
    print("oel")

In [197]:
def big_preprocess_lights(light_alarms: pd.DataFrame, predicting_min_date: str = None, predicting_max_date: str = None, for_predicting: bool = False) -> pd.DataFrame:
    """
    The function converts the data of the alarms into a dataframe that is usable for training the models.
    Review the coments on the code to understand step by step what the code does.
    """

    start_time = time.time()

    # We create a dataframe of each one of the ids and we will clean the alarms. The final objective is to calculate
    # the percentage of time the lampost is not working wich is the time that passes from an alarm "on" utill it is 
    # turned "off". In the dataframe we have cases in wich there are two consecutive "on" or "off" alarms so we have
    # filter and just keep the first "on" and the first "off" in this case so we get the real time the lampost has been
    # not functioning. In the case that the alarm is turned "on" during a week and it is not turned "off" untill the next
    # week we will insert a fake "off" alarm at the last moment of the week and we will turn it "on" again in the first moment
    # of the following week.

    # The first step will be to dowload the data:
    # Read recipe inputs

    df = light_alarms.copy()

    # Sort by id and date of the alarm:
    df["dated"] = pd.to_datetime(df["dated"])
    df = df.sort_values(["id", "dated"])
    # As far as we know if an alarm is "set" it means it is "off" so to keep things simple we will sustitute the
    # "set" values with "off"
    df.replace(to_replace="set", value="off", inplace=True)

    # Generate all the weeks from the first day we have data untill the last day. We will need this weeks later to
    # do a left join with the weeks that there are alarms

    # If we are preprocessing the data for doing predictions we will want to add the date_min and date max manualy,
    # otherwise the function will only consider the dates of the errors and you may end up with nan rows
    if for_predicting: 
        start_date = pd.to_datetime(predicting_min_date)
        end_date = pd.to_datetime(predicting_max_date)
    else:
        start_date = df["dated"].min()
        end_date = return_monday(df["dated"].max()) + pd.Timedelta(days=7)
    
    weeks = pd.date_range(start=start_date, end=end_date, freq='W').floor("D").strftime('%Y-%m-%d %H:%M:%S').tolist()
    # Transform to a dataframe to do operations later:
    weeks = pd.DataFrame(weeks, columns=["week"])
    weeks["week"] = pd.to_datetime(weeks["week"])
    weeks["week"] = weeks["week"].apply(lambda x: return_monday(x))
    # For each unique id we will implement all the code:
    # First create the dataframe where we will store all the resturned data:
    general_lag_dataframe = pd.DataFrame()

    # List of all the ids:
    ids_list = df["id"].unique()

    # Store the name of the columns in oder to later reordenate the final returned dataframe.
    if for_predicting:
        columns_order = ["id", "week-4", "hours_week-4", "week-3", "hours_week-3", "week-2", "hours_week-2", "week-1", "hours_week-1",
                        "current_week", "hours_current_week"]
    else:     
        columns_order = ["id", "week-4", "hours_week-4", "week-3", "hours_week-3", "week-2", "hours_week-2", "week-1", "hours_week-1",
                        "current_week", "hours_current_week", "week+1", "hours_week+1", "week+2", "hours_week+2", "week+3", "hours_week+3", "week+4", "hours_week+4"]

    # Generate a dataframe for each one of the ids and then apply all the transformations. Once done, add the data
    # to the dataframe general_lag_dataframe.
    for idd in ids_list:
        print(idd)
        
        # Dataframe of all the alarms for the id "idd":
        tt = df.loc[df["id"] == idd]
        
        #For each one of the elements in the list we have: elem[0] contains the week represented by sunday 
        # and elem[1] has the data in a dataframe
        grouped_weeks = list(tt.groupby(pd.Grouper(key="dated", freq="W")))
        # Here we will transform all the tuples to lists so we can change the elements after
        grouped_weeks = [list(elem) for elem in grouped_weeks]

        # Now we have to filtrate the empty dataframes that generates groupby for the weeks where there are no alarms
        grouped_weeks = [elem for elem in grouped_weeks if not elem[1].empty]

        # Eliminate the repeated flags such as "off" followed by "off" or "on" followed by "on" for the first element
        # of the list grouped_weeks. We do it because in the iteration we will not consider it so we have to do it now
        first_week_data = grouped_weeks[0][1]
        first_week_data["prev_flag"] = first_week_data["flag"].shift()

        first_week_data = first_week_data.loc[
            ((first_week_data["flag"] == "on") & (first_week_data["prev_flag"] == "off")) |
            ((first_week_data["flag"] == "off") & (first_week_data["prev_flag"] == "on")) |
            ((first_week_data["flag"] == "off") & (first_week_data["prev_flag"].isna())) | # This two last comparisons are for the first rows because they don't have a previous flag
            ((first_week_data["flag"] == "on") & (first_week_data["prev_flag"].isna()))
        ]

        # We store it to the data of the first week:
        grouped_weeks[0][1] = first_week_data

        # Here we begin the iteration for all the other weeks:
        for i, (week, data) in enumerate(grouped_weeks[1:], 1): # We do not consider the first week because it has no previous week to check
            # Get the data and week from the previous week:
            previous_week, previous_week_data = grouped_weeks[i-1][0], grouped_weeks[i-1][1]

            # Eliminate the repeated flags such as "off" followed by "off" or "on" followed by "on"
            data["prev_flag"] = data["flag"].shift()
            data = data.loc[
                ((data["flag"] == "on") & (data["prev_flag"] == "off")) |
                ((data["flag"] == "off") & (data["prev_flag"] == "on")) |
                ((data["flag"] == "off") & (data["prev_flag"].isna())) | # This two last comparisons are for the first rows because they don't have a previous flag
                ((data["flag"] == "on") & (data["prev_flag"].isna()))
            ]

            # Data that we will need:
            last_moment_previous_week = previous_week + pd.Timedelta(hours=23, minutes=59, seconds=59) #We will need the last moment of the week
            first_moment_current_week = (week - pd.Timedelta(days=week.dayofweek)).replace(hour=0, minute=0, second=0) #We will need to the first moment of the current week

            #We have to get the last flag of the previous week
            last_flag_previous_week = previous_week_data.loc[previous_week_data.index[-1], "flag"]

            if last_flag_previous_week == "on":
                # If the last flag from the previous week is "on" then we have to set a new row on the previous week data
                # in the last position to set a flag "off". Then, in the current week data we will add a new row before the
                # first week to set again the alarm to "on"

                # Here we create the new row to set the alarm "off" in the previous week
                new_row_previous_week = pd.DataFrame(
                    {
                    "id": [idd],
                    "dated": [last_moment_previous_week],
                    "alarm": ["turn_off_end_week"], # We will put this in alarm to know witch alarms where inserted by us
                    "flag": ["off"],
                    "prev_flag": ["Empty"] # Empty because we do not need it anymore and it will help us identify this rows
                    }
                )

                # We create the new df with the new row at the very end of the week
                new_previous_week_data = pd.concat(
                    [previous_week_data, new_row_previous_week],
                    sort=True # Remove the warning of pd.concat
                )
                # We update the dataframe in the list grouped_weeks
                grouped_weeks[i-1][1] = new_previous_week_data

                # Now we have to set the flag "on" in the first moment of the current week:
                # Here we create the new row to set the alarm "on" in the current week
                new_row_current_week = pd.DataFrame(
                    {
                    "id": [idd],
                    "dated": [first_moment_current_week],
                    "alarm": ["turn_on_begining_week"], # We will put this in alarm to know witch alarms where inserted by us
                    "flag": ["on"],
                    "prev_flag": ["Empty"] # Empty because we do not need it anymore and it will help us identify this rows
                    }
                )

                # We create the new df with the new row at the very begining of the week
                new_current_week_data = pd.concat(
                    [new_row_current_week, data],
                    sort=True # Remove the warning of pd.concat
                )
                # We update the dataframe in the list grouped_weeks
                grouped_weeks[i][1] = new_current_week_data
            else:
                # Simply update the dataframe with the same but with removed rows that contain two identical flags in a row
                grouped_weeks[i][1] = data

        # Once this is done we have to check if the last alarm of the last week is "on". In this case we will add a row turning it off
        # in the last moment of the week:
        last_recorded_week, last_recorded_week_data = grouped_weeks[-1][0], grouped_weeks[-1][1]

        # Get the last flag from the last week
        last_flag = last_recorded_week_data["flag"].values[-1]
        # and get the last moment of the last week
        last_moment_last_recorded_week = last_recorded_week + pd.Timedelta(hours=23, minutes=59, seconds=59)

        if last_flag == "on":
            new_row_last_flag = pd.DataFrame(
                {
                "id": [idd],
                "dated": [last_moment_last_recorded_week],
                "alarm": ["turn_off_end_last_week"], # We will put this in alarm to know witch alarms where inserted by us
                "flag": ["off"],
                "prev_flag": ["Empty"] # Empty because we do not need it anymore and it will help us identify this rows
                }
            )

            last_recorded_week_data = pd.concat(
                [last_recorded_week_data, new_row_last_flag],
                sort=True
            )

            grouped_weeks[-1][1] = last_recorded_week_data

        # Now we have to concat all the dataframes from all the weeks contained in the list grouped_weeks in one big dataframe
        concatenated_weeks = pd.concat(
            [week_data[1] for week_data in grouped_weeks],
            sort=True # Remove the warning of pd.concat
        )

        # At this point there are some cases where we will still have two "on" alarms or two "off" alarms in a row. For example
        # in the case that we have two weeks in a row where we only have "on" alarms utill now the code is going to return the 
        # begining of the end of the previous week with "off", the beggining of the current week with "off" and before the "off"
        # of the end of the week we will still have an "on" alarm. This is caused because the deletion of the same alarms in a row is done 
        # before the add of the new rows in the beggining and end of the week

        # So let's eliminate this cases too:
        # first we have to frop the old prev_flag column that now is useless:
        concatenated_weeks.drop("prev_flag", axis=1, inplace=True)

        # and create the new one:
        concatenated_weeks["prev_flag_concat"] = concatenated_weeks["flag"].shift()

        concatenated_weeks = concatenated_weeks.loc[
            ((concatenated_weeks["flag"] == "on") & (concatenated_weeks["prev_flag_concat"] == "off")) |
            ((concatenated_weeks["flag"] == "off") & (concatenated_weeks["prev_flag_concat"] == "on")) |
            ((concatenated_weeks["flag"] == "off") & (concatenated_weeks["prev_flag_concat"].isna())) | # This two last comparisons are for the first rows because they don't have a previous flag
            ((concatenated_weeks["flag"] == "on") & (concatenated_weeks["prev_flag_concat"].isna()))
        ]

        # We have some weeks where there is no data and thus if this weeks are between an "on" and "off" flag they will not
        # appear on the dataframe. What we will do is a left join with the variable week generated at the begining
        # of the notebook to see the rows that do not appear. Then we will activate the alarm at the begining of the week
        # and deactivate it at the end of the same week:

        # Set the day to monday
        concatenated_weeks["week"] = concatenated_weeks["dated"].apply(lambda x: return_monday(x))
        # Left join with weeks to detect the missing weeks
        concatenated_weeks_merged = pd.merge(weeks, concatenated_weeks, on="week", how="left")

        # Store the result of the filling:
        filled_dataframe = pd.DataFrame()

        # We add the first row
        if not pd.isna(concatenated_weeks_merged.iloc[0]["alarm"]):
            first_new_row = pd.DataFrame(
                {
                    "week": [concatenated_weeks_merged.iloc[0]["week"]],
                    "alarm": [concatenated_weeks_merged.iloc[0]["alarm"]],
                    "dated": [concatenated_weeks_merged.iloc[0]["dated"]],
                    "flag": [concatenated_weeks_merged.iloc[0]["flag"]],
                    "id": [concatenated_weeks_merged.iloc[0]["id"]]
                }
            )

            filled_dataframe = pd.concat(
                [filled_dataframe, first_new_row],
                sort=True
            )

        # Begin the iteration where we will add one by one the rows to the dataframe filled_dataframe
        for i in range(1, len(concatenated_weeks_merged)-1):

            current_row = concatenated_weeks_merged.iloc[i]

            # If we find a normal row we add it to the dataframe
            if (current_row["alarm"] == "lightcomm") | (current_row["alarm"] == "lighterr") | (current_row["alarm"] == "turn_off_end_week") | (current_row["alarm"] == "turn_on_begining_week"):
                current_row["dated"] = pd.to_datetime(current_row["dated"])

                new_row = pd.DataFrame(
                    {
                        "week": [current_row["week"]],
                        "alarm": [current_row["alarm"]],
                        "dated": [current_row["dated"]],
                        "flag": [current_row["flag"]],
                        "id": [current_row["id"]]
                    }
                )
                filled_dataframe = pd.concat([filled_dataframe, new_row], sort=True)

            if not filled_dataframe.empty:
                # If the last row of the filled dataframe is a "turn_off_week" ant the current is a Emty we add the "on" and
                # off for the begining and the end of the week
                last_row_filled_dataframe = filled_dataframe.iloc[-1]

                if (pd.isna(current_row["alarm"])) & (last_row_filled_dataframe["alarm"] in ["turn_off_end_week", "turn_off_end_week_filled"]):

                    new_row_begining_dated = datetime.datetime.combine(current_row["week"], datetime.time(0,0,0))
                    new_row_begining_week = pd.DataFrame(
                        {
                            "week": [current_row["week"]],
                            "alarm": ["turn_on_begining_week_filled"],
                            "dated": [new_row_begining_dated], # Add the time to the date
                            "flag": ["on"],
                            "id": [idd]
                        }
                    )
                    # We have to add 6 days because the representative of the week in this case is 
                    new_row_end_dated = datetime.datetime.combine(current_row["week"]+ pd.Timedelta(days=6), datetime.time(23,59,59))
                    new_row_end_week = pd.DataFrame(
                        {
                            "week": [current_row["week"]],
                            "alarm": ["turn_off_end_week_filled"],
                            "dated": [new_row_end_dated], # In this case we add the time to represent the last moment of the week
                            "flag": ["off"],
                            "id": [idd]
                        }
                    )

                    filled_dataframe = pd.concat(
                        [filled_dataframe, new_row_begining_week, new_row_end_week],
                        sort=True
                    )


        # Again we have to do a left join with the dataframe weeks to detect the NaN values:
        filled_dataframe_merged = pd.merge(weeks, filled_dataframe, on="week", how="left")

        # At this point there are some cases where the last alarm of the dataframe is an "on" so we have to turn the alarm off at the end of the week:
        if filled_dataframe_merged["flag"].iloc[-1] == "on": # Check if the last alarm is still on
            
            # get the last moment of the week
            end_of_week = pd.to_datetime(filled_dataframe_merged["week"].iloc[-1]) + pd.Timedelta(days=6, hours=23, minutes=59, seconds=59)

            row_to_add = pd.DataFrame(
                {
                    "week": [filled_dataframe_merged["week"].iloc[-1]],
                    "alarm": ["turn_off_end_week"],
                    "dated": end_of_week,
                    "flag": ["off"],
                    "id": idd
                }
            )

            filled_dataframe_merged = pd.concat(
                [filled_dataframe_merged, row_to_add],
                sort=True
            )
        
        # Now for each one of the weeks we have to calculate the total time passed between an "on" alarm and an "off" alarm

        # In this dataframe we will store the amount of hours of a light that has been malfunctioning for each week
        week_hours_dataframe = pd.DataFrame()
        for week in weeks["week"]:
            # In this variable we will store the amount of hours for this week:
            total_hours = 0
            on_timestamp = None

            # Dataframe with the alarms of the week:
            week_alarms_dataframe = filled_dataframe_merged.loc[filled_dataframe_merged["week"] == week]
            # Iterate trough the df to count the hours:
            for _, row in week_alarms_dataframe.iterrows():
                if row["flag"] == "on":
                    on_timestamp = row["dated"]

                if (row["flag"] == "off") & (on_timestamp is not None):
                    total_hours += (row["dated"] - on_timestamp).total_seconds() / 3600

                    on_timestamp = None

            new_week_hours = pd.DataFrame(
                {
                    "id": [idd],
                    "week": [week],
                    "malfunctioning_hours": [total_hours] 
                }
            )

            week_hours_dataframe = pd.concat(
                [week_hours_dataframe, new_week_hours],
                sort=True
            )

        # Now we want to get the data in the format:
        # row = {"week-4": date_week_prev_4, "hours_week-4": hours_week_prev_4, ..., "week-1": date_week_prev_1, "hours_week-1": hours_week_prev_1, "current_week": date_current_week, "hours_current_week":  "week+1": date_week_next_1, "hours_week+1": hours_week_next_1 ..., "week+4": date_week_next_4, "hours_week+4": hours_week_next_4}

        # In this dataframe we will store the data in the format we have mentioned:
        lag_dataframe = pd.DataFrame()
        
        # Here we have to follow two different paths. The first one will be when preparing the data for training and the other one will be when 
        # preparing the data for predicting. We shall begin with predicting:

        if for_predicting:
            lag_dataframe = pd.DataFrame(
                {
                    "id": [idd],

                    "week-4": [week_hours_dataframe.iloc[-4]["week"]],
                    "hours_week-4": [week_hours_dataframe.iloc[-4]["malfunctioning_hours"]],

                    "week-3": [week_hours_dataframe.iloc[-3]["week"]],
                    "hours_week-3": [week_hours_dataframe.iloc[-3]["malfunctioning_hours"]],

                    "week-2": [week_hours_dataframe.iloc[-2]["week"]],
                    "hours_week-2": [week_hours_dataframe.iloc[-2]["malfunctioning_hours"]],

                    "week-1": [week_hours_dataframe.iloc[-1]["week"]],
                    "hours_week-1": [week_hours_dataframe.iloc[-1]["malfunctioning_hours"]],

                    "current_week": [week_hours_dataframe.iloc[0]["week"]],
                    "hours_current_week": [week_hours_dataframe.iloc[0]["malfunctioning_hours"]]
                }
            )

            # Reorder the dataframe so it is in the same order we have defined:
            lag_dataframe = lag_dataframe[columns_order]
            
            # Add the dataframe to the general one:
            general_lag_dataframe = pd.concat(
                [general_lag_dataframe, lag_dataframe],
                sort=True
            )
        
        else:
            # Begin the loop at 4 and end at -4 so we don't get the error: "Out of range"
            for i in range(4, len(weeks)-4):
                
                # Create the new row to add:
                to_add_row = pd.DataFrame(
                    {
                        "id": [idd],

                        "week-4": [week_hours_dataframe.iloc[i-4]["week"]],
                        "hours_week-4": [week_hours_dataframe.iloc[i-4]["malfunctioning_hours"]],

                        "week-3": [week_hours_dataframe.iloc[i-3]["week"]],
                        "hours_week-3": [week_hours_dataframe.iloc[i-3]["malfunctioning_hours"]],

                        "week-2": [week_hours_dataframe.iloc[i-2]["week"]],
                        "hours_week-2": [week_hours_dataframe.iloc[i-2]["malfunctioning_hours"]],

                        "week-1": [week_hours_dataframe.iloc[i-1]["week"]],
                        "hours_week-1": [week_hours_dataframe.iloc[i-1]["malfunctioning_hours"]],

                        "current_week": [week_hours_dataframe.iloc[i]["week"]],
                        "hours_current_week": [week_hours_dataframe.iloc[i]["malfunctioning_hours"]],

                        "week+1": [week_hours_dataframe.iloc[i+1]["week"]],
                        "hours_week+1": [week_hours_dataframe.iloc[i+1]["malfunctioning_hours"]],

                        "week+2": [week_hours_dataframe.iloc[i+2]["week"]],
                        "hours_week+2": [week_hours_dataframe.iloc[i+2]["malfunctioning_hours"]],

                        "week+3": [week_hours_dataframe.iloc[i+3]["week"]],
                        "hours_week+3": [week_hours_dataframe.iloc[i+3]["malfunctioning_hours"]],

                        "week+4": [week_hours_dataframe.iloc[i+4]["week"]],
                        "hours_week+4": [week_hours_dataframe.iloc[i+4]["malfunctioning_hours"]]
                    }
                )

                lag_dataframe = pd.concat(
                    [lag_dataframe, to_add_row],
                    sort=True,
                    ignore_index=True
                )

            # Reorder the dataframe so it is in the same order we have defined:
            lag_dataframe = lag_dataframe[columns_order]
            
            # Add the dataframe to the general one:
            general_lag_dataframe = pd.concat(
                [general_lag_dataframe, lag_dataframe],
                sort=True
            )
        
    # Reordenate with the list columns_order:
    general_lag_dataframe = general_lag_dataframe[columns_order]

    end_time = time.time()

    print("Execution time:" + str(end_time - start_time))

    return general_lag_dataframe

def big_preprocess_eboxes(eboxes_alarms: pd.DataFrame, predicting_min_date: str = None, predicting_max_date: str = None, for_predicting: bool = False) -> pd.DataFrame:
    """
    The function converts the data of the alarms into a dataframe that is usable for training the models.
    Review the coments on the code to understand step by step what the code does.
    """
    
    start_time = time.time()

    # We create a dataframe of each one of the ids and we will clean the alarms. The final objective is to calculate
    # the percentage of time the ebox is not working wich is the time that passes from an alarm "off" utill it is 
    # turned "on". In the dataframe we have cases in wich there are two consecutive "on" or "off" alarms so we have
    # filter and just keep the first "on" and the first "off" in this case so we get the real time the ebox has been
    # not functioning. In the case that the alarm is turned "on" during a week and it is not turned "off" untill the next
    # week we will insert a fake "off" alarm at the last moment of the week and we will turn it "on" again in the first moment
    # of the following week.

    # The first step will be to dowload the data:
    # Read recipe inputs
    df = eboxes_alarms.copy()

    # We will reuse the code of the light alarms for the eboxes: The main thing that we have to modify for the eboxes
    # is that for the alarm subtype brdpower the flag "off" means that the ebox is suffering a breakdown and untill the
    # alarm turns "on" we will consider that the ebox has been down. This is exactly opposite to what happened with the
    # lamposts.

    # The first step is to change the name of the column subtype. Even tough this columns means a subtype of alarm, in this
    # code and from now on we will rename it as alarm so we are able to replicate a similar code as the one used for the lights
    df.rename(columns={"subtype": "alarm"}, inplace=True)

    # There are four different flags in this case: {off, on, offM, onM} depending if the error has been detected digitaly or manualy.
    # we do not care so we fill change this categories to {on off}:
    df["flag"] = df["flag"].replace(["onM"], "on")
    df["flag"] = df["flag"].replace(["offM"], "off")

    # As we already said the flags in the eboxes are opposite as the one in the lamposts, so to be able to use the same
    # code we will just change the values "off" to "on" and the "on" to "off" and then apply the same exact code that
    # we used for the lamposts.
    df["flag"] = df["flag"].replace({'on': 'offf', 'off': 'on'}).replace({"offf": "off"})

    # Sort by id and date of the alarm:
    df["dated"] = pd.to_datetime(df["dated"])
    df = df.sort_values(["id", "dated"])

    # Generate all the weeks from the first day we have data untill the last day. We will need this weeks later to
    # do a left join with the weeks that there are alarms

    # If we are preprocessing the data for doing predictions we will want to add the date_min and date max manualy,
    # otherwise the function will only consider the dates of the errors and you may end up with nan rows
    if for_predicting: 
        start_date = pd.to_datetime(predicting_min_date)
        end_date = pd.to_datetime(predicting_max_date)
    else:
        start_date = df["dated"].min()
        end_date = return_monday(df["dated"].max()) + pd.Timedelta(days=7)

    weeks = pd.date_range(start=start_date, end=end_date, freq='W').floor("D").strftime('%Y-%m-%d %H:%M:%S').tolist()
    # Transform to a dataframe to do operations later:
    weeks = pd.DataFrame(weeks, columns=["week"])
    weeks["week"] = pd.to_datetime(weeks["week"])
    weeks["week"] = weeks["week"].apply(lambda x: return_monday(x))

    # For each unique id we will implement all the code:
    # First create the dataframe where we will store all the resturned data:
    general_lag_dataframe = pd.DataFrame()

    # List of all the ids:
    ids_list = df["id"].unique()

    # Store the name of the columns in oder to later reordenate the final returned dataframe.
    if for_predicting:
        columns_order = ["id", "week-4", "hours_week-4", "week-3", "hours_week-3", "week-2", "hours_week-2", "week-1", "hours_week-1",
                        "current_week", "hours_current_week"]
    else:     
        columns_order = ["id", "week-4", "hours_week-4", "week-3", "hours_week-3", "week-2", "hours_week-2", "week-1", "hours_week-1",
                        "current_week", "hours_current_week", "week+1", "hours_week+1", "week+2", "hours_week+2", "week+3", "hours_week+3", "week+4", "hours_week+4"]

    # Generate a dataframe for each one of the ids and then apply all the transformations. Once done, add the data
    # to the dataframe general_lag_dataframe.
    for idd in ids_list:
        print(idd)
        
        # Dataframe of all the alarms for the id "idd":
        tt = df.loc[df["id"] == idd]
        
        #For each one of the elements in the list we have: elem[0] contains the week represented by sunday 
        # and elem[1] has the data in a dataframe
        grouped_weeks = list(tt.groupby(pd.Grouper(key="dated", freq="W")))
        # Here we will transform all the tuples to lists so we can change the elements after
        grouped_weeks = [list(elem) for elem in grouped_weeks]

        # Now we have to filtrate the empty dataframes that generates groupby for the weeks where there are no alarms
        grouped_weeks = [elem for elem in grouped_weeks if not elem[1].empty]

        # Eliminate the repeated flags such as "off" followed by "off" or "on" followed by "on" for the first element
        # of the list grouped_weeks. We do it because in the iteration we will not consider it so we have to do it now
        first_week_data = grouped_weeks[0][1]
        first_week_data["prev_flag"] = first_week_data["flag"].shift()

        first_week_data = first_week_data.loc[
            ((first_week_data["flag"] == "on") & (first_week_data["prev_flag"] == "off")) |
            ((first_week_data["flag"] == "off") & (first_week_data["prev_flag"] == "on")) |
            ((first_week_data["flag"] == "off") & (first_week_data["prev_flag"].isna())) | # This two last comparisons are for the first rows because they don't have a previous flag
            ((first_week_data["flag"] == "on") & (first_week_data["prev_flag"].isna()))
        ]

        # We store it to the data of the first week:
        grouped_weeks[0][1] = first_week_data

        # Here we begin the iteration for all the other weeks:
        for i, (week, data) in enumerate(grouped_weeks[1:], 1): # We do not consider the first week because it has no previous week to check
            # Get the data and week from the previous week:
            previous_week, previous_week_data = grouped_weeks[i-1][0], grouped_weeks[i-1][1]

            # Eliminate the repeated flags such as "off" followed by "off" or "on" followed by "on"
            data["prev_flag"] = data["flag"].shift()
            data = data.loc[
                ((data["flag"] == "on") & (data["prev_flag"] == "off")) |
                ((data["flag"] == "off") & (data["prev_flag"] == "on")) |
                ((data["flag"] == "off") & (data["prev_flag"].isna())) | # This two last comparisons are for the first rows because they don't have a previous flag
                ((data["flag"] == "on") & (data["prev_flag"].isna()))
            ]

            # Data that we will need:
            last_moment_previous_week = previous_week + pd.Timedelta(hours=23, minutes=59, seconds=59) #We will need the last moment of the week
            first_moment_current_week = (week - pd.Timedelta(days=week.dayofweek)).replace(hour=0, minute=0, second=0) #We will need to the first moment of the current week

            #We have to get the last flag of the previous week
            last_flag_previous_week = previous_week_data.loc[previous_week_data.index[-1], "flag"]

            if last_flag_previous_week == "on":
                # If the last flag from the previous week is "on" then we have to set a new row on the previous week data
                # in the last position to set a flag "off". Then, in the current week data we will add a new row before the
                # first week to set again the alarm to "on"

                # Here we create the new row to set the alarm "off" in the previous week
                new_row_previous_week = pd.DataFrame(
                    {
                    "id": [idd],
                    "dated": [last_moment_previous_week],
                    "alarm": ["turn_off_end_week"], # We will put this in alarm to know witch alarms where inserted by us
                    "flag": ["off"],
                    "prev_flag": ["Empty"] # Empty because we do not need it anymore and it will help us identify this rows
                    }
                )

                # We create the new df with the new row at the very end of the week
                new_previous_week_data = pd.concat(
                    [previous_week_data, new_row_previous_week],
                    sort=True # Remove the warning of pd.concat
                )
                # We update the dataframe in the list grouped_weeks
                grouped_weeks[i-1][1] = new_previous_week_data

                # Now we have to set the flag "on" in the first moment of the current week:
                # Here we create the new row to set the alarm "on" in the current week
                new_row_current_week = pd.DataFrame(
                    {
                    "id": [idd],
                    "dated": [first_moment_current_week],
                    "alarm": ["turn_on_begining_week"], # We will put this in alarm to know witch alarms where inserted by us
                    "flag": ["on"],
                    "prev_flag": ["Empty"] # Empty because we do not need it anymore and it will help us identify this rows
                    }
                )

                # We create the new df with the new row at the very begining of the week
                new_current_week_data = pd.concat(
                    [new_row_current_week, data],
                    sort=True # Remove the warning of pd.concat
                )
                # We update the dataframe in the list grouped_weeks
                grouped_weeks[i][1] = new_current_week_data
            else:
                # Simply update the dataframe with the same but with removed rows that contain two identical flags in a row
                grouped_weeks[i][1] = data

        # Once this is done we have to check if the last alarm of the last week is "on". In this case we will add a row turning it off
        # in the last moment of the week:
        last_recorded_week, last_recorded_week_data = grouped_weeks[-1][0], grouped_weeks[-1][1]

        # Get the last flag from the last week
        last_flag = last_recorded_week_data["flag"].values[-1]
        # and get the last moment of the last week
        last_moment_last_recorded_week = last_recorded_week + pd.Timedelta(hours=23, minutes=59, seconds=59)

        if last_flag == "on":
            new_row_last_flag = pd.DataFrame(
                {
                "id": [idd],
                "dated": [last_moment_last_recorded_week],
                "alarm": ["turn_off_end_last_week"], # We will put this in alarm to know witch alarms where inserted by us
                "flag": ["off"],
                "prev_flag": ["Empty"] # Empty because we do not need it anymore and it will help us identify this rows
                }
            )

            last_recorded_week_data = pd.concat(
                [last_recorded_week_data, new_row_last_flag],
                sort=True
            )

            grouped_weeks[-1][1] = last_recorded_week_data

        # Now we have to concat all the dataframes from all the weeks contained in the list grouped_weeks in one big dataframe
        concatenated_weeks = pd.concat(
            [week_data[1] for week_data in grouped_weeks],
            sort=True # Remove the warning of pd.concat
        )

        # At this point there are some cases where we will still have two "on" alarms or two "off" alarms in a row. For example
        # in the case that we have two weeks in a row where we only have "on" alarms utill now the code is going to return the 
        # begining of the end of the previous week with "off", the beggining of the current week with "off" and before the "off"
        # of the end of the week we will still have an "on" alarm. This is caused because the deletion of the same alarms in a row is done 
        # before the add of the new rows in the beggining and end of the week

        # So let's eliminate this cases too:
        # first we have to frop the old prev_flag column that now is useless:
        concatenated_weeks.drop("prev_flag", axis=1, inplace=True)

        # and create the new one:
        concatenated_weeks["prev_flag_concat"] = concatenated_weeks["flag"].shift()

        concatenated_weeks = concatenated_weeks.loc[
            ((concatenated_weeks["flag"] == "on") & (concatenated_weeks["prev_flag_concat"] == "off")) |
            ((concatenated_weeks["flag"] == "off") & (concatenated_weeks["prev_flag_concat"] == "on")) |
            ((concatenated_weeks["flag"] == "off") & (concatenated_weeks["prev_flag_concat"].isna())) | # This two last comparisons are for the first rows because they don't have a previous flag
            ((concatenated_weeks["flag"] == "on") & (concatenated_weeks["prev_flag_concat"].isna()))
        ]

        # We have some weeks where there is no data and thus if this weeks are between an "on" and "off" flag they will not
        # appear on the dataframe. What we will do is a left join with the variable week generated at the begining
        # of the notebook to see the rows that do not appear. Then we will activate the alarm at the begining of the week
        # and deactivate it at the end of the same week:

        # Set the day to monday
        concatenated_weeks["week"] = concatenated_weeks["dated"].apply(lambda x: return_monday(x))
        # Left join with weeks to detect the missing weeks
        concatenated_weeks_merged = pd.merge(weeks, concatenated_weeks, on="week", how="left")

        # Store the result of the filling:
        filled_dataframe = pd.DataFrame()

        # We add the first row
        if not pd.isna(concatenated_weeks_merged.iloc[0]["alarm"]):
            first_new_row = pd.DataFrame(
                {
                    "week": [concatenated_weeks_merged.iloc[0]["week"]],
                    "alarm": [concatenated_weeks_merged.iloc[0]["alarm"]],
                    "dated": [concatenated_weeks_merged.iloc[0]["dated"]],
                    "flag": [concatenated_weeks_merged.iloc[0]["flag"]],
                    "id": [concatenated_weeks_merged.iloc[0]["id"]]
                }
            )

            filled_dataframe = pd.concat(
                [filled_dataframe, first_new_row],
                sort=True
            )

        # Begin the iteration where we will add one by one the rows to the dataframe filled_dataframe
        for i in range(1, len(concatenated_weeks_merged)-1):

            current_row = concatenated_weeks_merged.iloc[i]

            # If we find a normal row we add it to the dataframe
            if (current_row["alarm"] == "brdpower") | (current_row["alarm"] == "turn_off_end_week") | (current_row["alarm"] == "turn_on_begining_week"):
                current_row["dated"] = pd.to_datetime(current_row["dated"])

                new_row = pd.DataFrame(
                    {
                        "week": [current_row["week"]],
                        "alarm": [current_row["alarm"]],
                        "dated": [current_row["dated"]],
                        "flag": [current_row["flag"]],
                        "id": [current_row["id"]]
                    }
                )
                filled_dataframe = pd.concat([filled_dataframe, new_row], sort=True)

            if not filled_dataframe.empty:
                # If the last row of the filled dataframe is a "turn_off_week" ant the current is a Emty we add the "on" and
                # off for the begining and the end of the week
                last_row_filled_dataframe = filled_dataframe.iloc[-1]

                if (pd.isna(current_row["alarm"])) & (last_row_filled_dataframe["alarm"] in ["turn_off_end_week", "turn_off_end_week_filled"]):

                    new_row_begining_dated = datetime.datetime.combine(current_row["week"], datetime.time(0,0,0))
                    new_row_begining_week = pd.DataFrame(
                        {
                            "week": [current_row["week"]],
                            "alarm": ["turn_on_begining_week_filled"],
                            "dated": [new_row_begining_dated], # Add the time to the date
                            "flag": ["on"],
                            "id": [idd]
                        }
                    )
                    # We have to add 6 days because the representative of the week in this case is 
                    new_row_end_dated = datetime.datetime.combine(current_row["week"]+ pd.Timedelta(days=6), datetime.time(23,59,59))
                    new_row_end_week = pd.DataFrame(
                        {
                            "week": [current_row["week"]],
                            "alarm": ["turn_off_end_week_filled"],
                            "dated": [new_row_end_dated], # In this case we add the time to represent the last moment of the week
                            "flag": ["off"],
                            "id": [idd]
                        }
                    )

                    filled_dataframe = pd.concat(
                        [filled_dataframe, new_row_begining_week, new_row_end_week],
                        sort=True
                    )

        # Again we have to do a left join with the dataframe weeks to detect the NaN values:
        filled_dataframe_merged = pd.merge(weeks, filled_dataframe, on="week", how="left")
        
        # Now for each one of the weeks we have to calculate the total time passed between an "on" alarm and an "off" alarm

        # In this dataframe we will store the amount of hours of a ebox that has been malfunctioning for each week
        week_hours_dataframe = pd.DataFrame()
        for week in weeks["week"]:
            # In this variable we will store the amount of hours for this week:
            total_hours = 0
            on_timestamp = None

            # Dataframe with the alarms of the week:
            week_alarms_dataframe = filled_dataframe_merged.loc[filled_dataframe_merged["week"] == week]
            # Iterate trough the df to count the hours:
            for _, row in week_alarms_dataframe.iterrows():
                if row["flag"] == "on":
                    on_timestamp = row["dated"]

                if (row["flag"] == "off") & (on_timestamp is not None):
                    total_hours += (row["dated"] - on_timestamp).total_seconds() / 3600

                    on_timestamp = None

            new_week_hours = pd.DataFrame(
                {
                    "id": [idd],
                    "week": [week],
                    "malfunctioning_hours": [total_hours] 
                }
            )

            week_hours_dataframe = pd.concat(
                [week_hours_dataframe, new_week_hours],
                sort=True
            )

        # Now we want to get the data in the format:
        # row = {"week-4": date_week_prev_4, "hours_week-4": hours_week_prev_4, ..., "week-1": date_week_prev_1, "hours_week-1": hours_week_prev_1, "current_week": date_current_week, "hours_current_week":  "week+1": date_week_next_1, "hours_week+1": hours_week_next_1 ..., "week+4": date_week_next_4, "hours_week+4": hours_week_next_4}

        # In this dataframe we will store the data in the format we have mentioned:
        lag_dataframe = pd.DataFrame()

        # Here we have to follow two different paths. The first one will be when preparing the data for training and the other one will be when 
        # preparing the data for predicting. We shall begin with predicting:

        if for_predicting:
            lag_dataframe = pd.DataFrame(
                {
                    "id": [idd],

                    "week-4": [week_hours_dataframe.iloc[-4]["week"]],
                    "hours_week-4": [week_hours_dataframe.iloc[-4]["malfunctioning_hours"]],

                    "week-3": [week_hours_dataframe.iloc[-3]["week"]],
                    "hours_week-3": [week_hours_dataframe.iloc[-3]["malfunctioning_hours"]],

                    "week-2": [week_hours_dataframe.iloc[-2]["week"]],
                    "hours_week-2": [week_hours_dataframe.iloc[-2]["malfunctioning_hours"]],

                    "week-1": [week_hours_dataframe.iloc[-1]["week"]],
                    "hours_week-1": [week_hours_dataframe.iloc[-1]["malfunctioning_hours"]],

                    "current_week": [week_hours_dataframe.iloc[0]["week"]],
                    "hours_current_week": [week_hours_dataframe.iloc[0]["malfunctioning_hours"]]
                }
            )

            # Reorder the dataframe so it is in the same order we have defined:
            lag_dataframe = lag_dataframe[columns_order]
            
            # Add the dataframe to the general one:
            general_lag_dataframe = pd.concat(
                [general_lag_dataframe, lag_dataframe],
                sort=True
            )

        else:
            # Begin the loop at 4 and end at -4 so we don't get the error: "Out of range"
            for i in range(4, len(weeks)-4):
                # Create the new row to add:
                to_add_row = pd.DataFrame(
                    {
                        "id": [idd],

                        "week-4": [week_hours_dataframe.iloc[i-4]["week"]],
                        "hours_week-4": [week_hours_dataframe.iloc[i-4]["malfunctioning_hours"]],

                        "week-3": [week_hours_dataframe.iloc[i-3]["week"]],
                        "hours_week-3": [week_hours_dataframe.iloc[i-3]["malfunctioning_hours"]],

                        "week-2": [week_hours_dataframe.iloc[i-2]["week"]],
                        "hours_week-2": [week_hours_dataframe.iloc[i-2]["malfunctioning_hours"]],

                        "week-1": [week_hours_dataframe.iloc[i-1]["week"]],
                        "hours_week-1": [week_hours_dataframe.iloc[i-1]["malfunctioning_hours"]],

                        "current_week": [week_hours_dataframe.iloc[i]["week"]],
                        "hours_current_week": [week_hours_dataframe.iloc[i]["malfunctioning_hours"]],

                        "week+1": [week_hours_dataframe.iloc[i+1]["week"]],
                        "hours_week+1": [week_hours_dataframe.iloc[i+1]["malfunctioning_hours"]],

                        "week+2": [week_hours_dataframe.iloc[i+2]["week"]],
                        "hours_week+2": [week_hours_dataframe.iloc[i+2]["malfunctioning_hours"]],

                        "week+3": [week_hours_dataframe.iloc[i+3]["week"]],
                        "hours_week+3": [week_hours_dataframe.iloc[i+3]["malfunctioning_hours"]],

                        "week+4": [week_hours_dataframe.iloc[i+4]["week"]],
                        "hours_week+4": [week_hours_dataframe.iloc[i+4]["malfunctioning_hours"]]
                    }
                )

                lag_dataframe = pd.concat(
                    [lag_dataframe, to_add_row],
                    sort=True,
                    ignore_index=True
                )

            # Reorder the dataframe so it is in the same order we have defined:
            lag_dataframe = lag_dataframe[columns_order]
            
            # Add the dataframe to the general one:
            general_lag_dataframe = pd.concat(
                [general_lag_dataframe, lag_dataframe],
                sort=True
            )
        
    # Reordenate with the list columns_order:
    general_lag_dataframe = general_lag_dataframe[columns_order]

    end_time = time.time()

    print("Execution time:" + str(end_time - start_time))

    return general_lag_dataframe

# Documentation predictive maintenance
This is the repository of the implementation of the predictive maintenance project for INNERGY. 

## Description of the project:
The main objective of the project is the creation of a machine learning model able to calculate accurate predictions of future breakdowns for both public luminaire (from now on lights) and electric panels (eboxes). The repo contains the whole pipeline from raw data preprocessing to training the models and doing predictions.

The model will take as input information about the breakdown history from the last 5 weeks of a light or an ebox together with context information such as electric readings data and weather conditions. The output is the probability of breakdown in the next 4 weeks.

The code has four main functionalities that can be combined depending on the necessities of the user:
* **Meteo web scrapping:** Gather the meteorological information from a certain municipality on a specified time period using web scraping
* **Data preprocessing:** Pipeline to do the necessary data preprocessing of the raw breakdown data, the raw readings data and the scrapped meteorological data
* **Model training:** Training of the machine learning models using the prerpocessed data
* **Doing predictions:** Gather data from the last 5 weeks, preprocess it and do predictions using the trained models

Let's explain step by step how each one of the modules work:

## Meteo web scrapping
Code in the file scrapy_meteo.py of the folder scrapy_meteo. As we have already mentioned this module gathers the raw meteorological context data from the website .
To exectute the scraper, navigate to the folder scrapy_meteo in the repo and run the following in the terminal:

```
python scrapy_meteo.py --di "/local/directory/folder" --mu municipalities_list --dr date_ranges --li list_of_links
```
Let's take a look at the arguments:
* ```--di``` is the local directory of the folder you want to store the data extracted using the scraper.
* ```--mu``` is a list of the names of the municipalities you want to get the data.
* ```--dr``` is a dictionary where the municipality names serve as keys, and the corresponding values are tuples containing the desired date ranges for data extraction. See the example below for clarification.
* ```--li``` is a list of the links for doing the scraping.

For example, in case that you want to gather meteo information from the Spanish municipalities of Illora, Mejorada and Canyelles you should run in the terminal:

```python scrapy_meteo.py --di "/home/leibniz/Desktop/IHMAN/meteo_raw_data" --mu illora mejorada canyelles --dr "{'illora': ('2015-01-01', '2023-04-01'), 'mejorada': ('2014-01-01', '2023-04-01'), 'canyelles': ('2015-01-01', '2023-04-01')}" --li "https://www.wunderground.com/history/monthly/es/íllora/LEMG/date/" "https://www.wunderground.com/history/monthly/es/mejorada-del-campo/IMEJOR1/date/" "https://www.wunderground.com/history/monthly/es/canyelles/ICANYE10/date/"```

The models have been trained with data from this municipalities and in this exact date ranges so if your objective is to gather the information for replicating the the training some of the arguments have implemented default values to make the syntaxis more clear, so runing the following will gather the same data as the previous command:

```python scrapy_meteo.py --di "/home/leibniz/Desktop/IHMAN/meteo_raw_data"```

It is frequent to get errors when running this code because of conexion issues. We recommend to try to run the code again if you get an error. If the error persists after 3 or 4 tries it will be better to try to get the information in separte dataframes by running one municiaplity at a time.

## Data preprocessing
The code in the file preprocessing_main.py executes the preprocessing of all the data and prepares it for the training phase. For runing the preprocessing you must have 3 subfolders in the same folder somewhere on your local machine. The first one is the folder **raw_data** where you must have the following files:
* municipality_eboxes_alarms.csv For example canyelles_eboxes_alarms.csv
* municipality_lights_alarms.csv For example canyelles_lights_alarms.csv
* municipality_nodes.csv This file conatains the information of the eboxes associated at each one of the lights. For example canyelles_nodes.csv
* municipality_readings_sorted.csv This file contains the readings powerActive, powerReactive, powerActivePeak, powerReactivePeak **sorted** by date. For example canyelles_readings_sorted.csv

The second folder is **meteo_raw_data** that must contain the file:
* new_meteo_municipality.csv This file contains the meteo data that is the output of the scraper. For example new_meteo_canyelles.csv

Note that you must have each one of this files for each one of the municipalities that you want to preprocess.

The third folder is **preprocessing_results**. Once you execute the module, the code will look for this folder to drop the prerprocessed dataframes

To execute the data preprocess run the following command in the terminal:
```
python preprocessing_main.py --di "/local/directory/folder" --mu municipalities_list --ws n_weeks
```
Let's take a look at the arguments:
* ```--di``` is the local directory of the folder where you must have the subfolders raw_data and meteo_raw_data.
* ```--mu``` is a list of the names of the municipalities you want to include in the data preprocessing.
* ```--ws``` is the number of weeks data that we include in each one of the batches when prerpocessing the dataframes of readings. If you get a Memory error consider making this argument smaller. It defaults as 40.

For example, in case that you want to preprocess data from the Spanish municipalities of Illora, Mejorada and Canyelles with a datasplit of 40 you should run in the terminal:

```
python preprocessing_main.py --di "/home/leibniz/Desktop/IHMAN" --mu illora mejorada canyelles --ws 40
```

## Model training
For training the model you have two options. The first one is to use the step by step code in the jupyter notebook model_train.ipynb where there is a clear explanation of each step of the process. If you choose this approach you will be able to execute grid searches and tune the hyperparameters of the model to better fit your training data. The other option is to train the model with a set of hyperparameters optimized to fit the training dataset composed of data from Illora, Mejorada and Canyelles. If you choose the last option run the following command in the terminal:

```
python --di "/local/directory/folder" --de device --st store --rew rewrite --mo model
```
Let's take a look at the arguments:
* ```--di``` is the local directory where you must have the folder preprocessing_results with the output of the preprocessing module.
* ```--de``` is the device for which you want to train a prediction model.
* ```--st``` is a boolean that indicates wether or not you want to store the trained model in the folder predictive_models in the repo
* ```--rew``` is a boolean that indicates wether or not you want to rewrite the default models that come with the repository
* ```--mo``` is the type of model you want to train in the case that you have set the argument ```--de``` to light. It is either "default" for a model that uses the electrical readings but is trained in less data or "adboc" for training the Ada Boost Combined Predictor which uses much more data but does not use the readings.

For example for training a lights breakdown prediction model storing the models without rewriting the models I would run in my machine:

```
python --di "/home/leibniz/Desktop/IHMAN" --de "lights" --st True --rew False --mo "adboc"
```
in case I want to train it for eboxes:
```
python --di "/home/leibniz/Desktop/IHMAN" --de "eboxes" --st True --rew False
```
